In [1]:
import requests
import pandas as pd
import numpy as np
import time
import pickle
import warnings
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


# Initialize the FMP data fetcher
api_key = "bEiVRux9rewQy16TXMPxDqBAQGIW8UBd"
base_url = "https://financialmodelingprep.com/api/v3"

start_date = "1996-01-01"
end_date = "2024-12-31"

# Create directory for storing CSV files if it doesn't exist
output_dir = "sp500_constituents"
os.makedirs(output_dir, exist_ok=True)

# Fetch current S&P 500 constituents
endpoint = f"{base_url}/sp500_constituent"
params = {
    "apikey": api_key
}

response = requests.get(endpoint, params=params)
if response.status_code == 200:
    constituents = response.json()
    
    # Convert to DataFrame
    df = pd.DataFrame(constituents)
    
    # Save to CSV with current date
    current_date = datetime.now().strftime("%Y%m%d")
    output_file = os.path.join(output_dir, f"sp500_constituents_{current_date}.csv")
    df.to_csv(output_file, index=False)
    print(f"Saved S&P 500 constituents to {output_file}")
    
    # Print summary
    print(f"\nTotal companies in S&P 500: {len(df)}")
    print("\nFirst few companies:")
    print(df[['symbol', 'name', 'sector']].head())
else:
    print(f"Error fetching data: {response.status_code}")

Saved S&P 500 constituents to sp500_constituents\sp500_constituents_20250511.csv

Total companies in S&P 500: 503

First few companies:
  symbol                      name                  sector
0   DASH                  DoorDash  Communication Services
1    EXE             Expand Energy                  Energy
2    TKO        TKO Group Holdings  Communication Services
3    WSM           Williams-Sonoma       Consumer Cyclical
4    APO  Apollo Global Management      Financial Services


In [2]:
# Read the CSV file
df = pd.read_csv('sp500_constituents/sp500_constituents_20250511.csv')  # Adjust the filename if needed

# Extract symbols column
symbols = df['symbol'].tolist()

# Save to pickle file
with open("sp500_symbols.pickle", "wb") as f:
    pickle.dump(symbols, f)
print(f"Saved {len(symbols)} symbols to sp500_symbols.pickle")

# Load and print the symbols from pickle file
with open("sp500_symbols.pickle", "rb") as f:
    loaded_symbols = pickle.load(f)

print("Loaded symbols from pickle file:")
print(f"Total symbols: {len(loaded_symbols)}")
print("\nFirst 20 symbols:")
print(loaded_symbols[:20])

Saved 503 symbols to sp500_symbols.pickle
Loaded symbols from pickle file:
Total symbols: 503

First 20 symbols:
['DASH', 'EXE', 'TKO', 'WSM', 'APO', 'LII', 'WDAY', 'TPL', 'DELL', 'ERIE', 'PLTR', 'SW', 'CRWD', 'GDDY', 'KKR', 'VST', 'GEV', 'SOLV', 'DECK', 'SMCI']


In [3]:
tickers_considered = loaded_symbols[:20]

print(tickers_considered)

['DASH', 'EXE', 'TKO', 'WSM', 'APO', 'LII', 'WDAY', 'TPL', 'DELL', 'ERIE', 'PLTR', 'SW', 'CRWD', 'GDDY', 'KKR', 'VST', 'GEV', 'SOLV', 'DECK', 'SMCI']


In [58]:
#downloading the stock data from tickers_considered and filter out the 
# Create directory for storing individual CSV files
output_dir = "stock_prices"
os.makedirs(output_dir, exist_ok=True)

insufficient_data_stocks = []

# Function to fetch historical prices
def fetch_historical_prices(symbol):
    endpoint = f"{base_url}/historical-price-full/{symbol}"
    params = {
        "from": start_date,
        "to": end_date,
        #"to": datetime.now().strftime("%Y-%m-%d")
        "apikey": api_key
    }
    
    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'historical' in data:
            df = pd.DataFrame(data['historical'])
            return df
    return None


# Fetch data for each ticker
for i, symbol in enumerate(tickers_considered):
    print(f'\n[{i+1}/{len(tickers_considered)}] Fetching close price for {symbol}...')
    df = fetch_historical_prices(symbol)
    
    if df is not None:
        # Save to individual CSV file
        csv_path = os.path.join(output_dir, f"{symbol}_prices.csv")
        df.to_csv(csv_path, index=False)

        if len(df) < 630:  # Check if data points are less than 630
            insufficient_data_stocks.append(symbol)
            print(f"Warning: {symbol} has only {len(df)} data points")
        else:
            print(f"Saved {len(df)} rows for {symbol}")

    # Add a small delay to avoid hitting API rate limits
    time.sleep(0.5)

# Print stocks with insufficient data
print("\nStocks with insufficient data (less than 630 rows):")
for stock in insufficient_data_stocks:
    print(f"{stock}: {len(pd.read_csv(os.path.join(output_dir, f'{stock}_prices.csv')))} rows")

# Create final_tickers by removing insufficient data stocks
final_tickers = [ticker for ticker in tickers_considered if ticker not in insufficient_data_stocks]

# Save final_tickers to pickle
with open("final_tickers.pickle", "wb") as f:
    pickle.dump(final_tickers, f)

# # Save the stock data dictionary
# with open("stock_data.pickle", "wb") as f:
#     pickle.dump(stock_data, f)

print(f"\nOriginal number of tickers: {len(tickers_considered)}")
print(f"Number of tickers with insufficient data: {len(insufficient_data_stocks)}")
print(f"Final number of tickers: {len(final_tickers)}")
print(final_tickers)

# Print summary of saved files
print("\nSummary of saved files:")
print(f"Total CSV files created: {len(os.listdir(output_dir))}")
print(f"Files are saved in: {os.path.abspath(output_dir)}")


[1/20] Fetching close price for DASH...
Saved 1021 rows for DASH

[2/20] Fetching close price for EXE...
Saved 978 rows for EXE

[3/20] Fetching close price for TKO...
Saved 6341 rows for TKO

[4/20] Fetching close price for WSM...
Saved 7300 rows for WSM

[5/20] Fetching close price for APO...
Saved 3462 rows for APO

[6/20] Fetching close price for LII...
Saved 6398 rows for LII

[7/20] Fetching close price for WDAY...
Saved 3035 rows for WDAY

[8/20] Fetching close price for TPL...
Saved 7300 rows for TPL

[9/20] Fetching close price for DELL...
Saved 2107 rows for DELL

[10/20] Fetching close price for ERIE...
Saved 7300 rows for ERIE

[11/20] Fetching close price for PLTR...
Saved 1070 rows for PLTR

[12/20] Fetching close price for SW...
Saved 2396 rows for SW

[13/20] Fetching close price for CRWD...
Saved 1399 rows for CRWD

[14/20] Fetching close price for GDDY...
Saved 2456 rows for GDDY

[15/20] Fetching close price for KKR...
Saved 4344 rows for KKR

[16/20] Fetching close

In [25]:
# Load final_tickers from pickle file
with open("final_tickers.pickle", "rb") as f:
    final_tickers = pickle.load(f)

#print(f"Total number of final tickers: {len(final_tickers)}")

# Create an empty list to store all data
all_data = []

# Read each stock's CSV file and append to the list
for symbol in final_tickers:
    # Read the CSV file
    file_path = os.path.join("stock_prices", f"{symbol}_prices.csv")
    df = pd.read_csv(file_path)

    # Add symbol column
    df['symbol'] = symbol

    # Append to the list
    all_data.append(df)

# Combine all data into a single DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Sort by date and symbol
combined_df = combined_df.sort_values(['date', 'symbol'], ascending=[False, True])
# Reorder columns to put 'symbol' as the second column
columns = combined_df.columns.tolist()
columns.remove('symbol')
columns.remove('date')
new_column_order = ['date', 'symbol'] + columns
combined_df = combined_df[new_column_order].reset_index(drop=True)

# Print summary statistics
print("\nSummary of the data:")
print(f"Total number of rows: {len(combined_df)}")
print(f"Number of unique stocks: {combined_df['symbol'].nunique()}")
print(f"Date range: from {combined_df['date'].min()} to {combined_df['date'].max()}")

# Display the first few rows
print("First few rows of the combined data:")
print(combined_df.head())


Summary of the data:
Total number of rows: 70751
Number of unique stocks: 18
Date range: from 1996-01-02 to 2024-12-31
First few rows of the combined data:
         date symbol    open    high     low   close  adjClose   volume  \
0  2024-12-31    APO  167.00  167.60  164.93  165.16    164.69  2817031   
1  2024-12-31   CRWD  351.86  351.86  339.72  342.16    342.16  2237245   
2  2024-12-31   DASH  169.62  170.08  167.27  167.75    167.75  1241521   
3  2024-12-31   DECK  206.58  207.00  203.04  203.09    203.09  1317389   
4  2024-12-31   DELL  115.33  116.09  114.16  115.24    114.05  3874700   

   unadjustedVolume  change  changePercent      vwap            label  \
0           2817031   -1.84      -1.100000  166.1725  December 31, 24   
1           2237245   -9.70      -2.760000  346.4000  December 31, 24   
2           1241521   -1.87      -1.100000  168.6800  December 31, 24   
3           1317389   -3.49      -1.690000  204.9275  December 31, 24   
4           3874700   -0.09

In [34]:
combined_df = combined_df[['date', 'symbol', 'open', 'high', 'low', 'close', 'volume']]

#Check the data for any stock
combined_df[combined_df["symbol"]=="WSM"].sort_values('date', ascending=False)

,date,symbol,open,high,low,close,volume
17,2024-12-31,WSM,186.17,188.34,183.43,185.18,996100
35,2024-12-30,WSM,184.15,188.84,183.04,186.51,2909195
53,2024-12-27,WSM,187.30,188.97,185.15,186.05,772200
71,2024-12-26,WSM,186.40,188.71,186.05,187.63,1227200
89,2024-12-24,WSM,186.55,188.43,185.79,187.73,586900
...,...,...,...,...,...,...,...
70734,1996-01-08,WSM,2.27,2.28,2.20,2.28,292800
70738,1996-01-05,WSM,2.27,2.27,2.20,2.27,2770400
70742,1996-01-04,WSM,2.30,2.30,2.20,2.27,1379200
70746,1996-01-03,WSM,2.38,2.38,2.22,2.30,1070400


In [39]:
# Create a copy of the DataFrame
combined_df2 = combined_df.copy()

# Convert date to datetime and set multi-index
combined_df2['date'] = pd.to_datetime(combined_df2['date'])
combined_df2.set_index(['date', 'symbol'], inplace=True)

# Sort the index
combined_df2.sort_index(ascending=[False, True], inplace=True)

display(combined_df2)

open       high        low      close   volume
date       symbol                                                     
2024-12-31 APO     167.00000  167.60000  164.93000  165.16000  2817031
           CRWD    351.86000  351.86000  339.72000  342.16000  2237245
           DASH    169.62000  170.08000  167.27000  167.75000  1241521
           DECK    206.58000  207.00000  203.04000  203.09000  1317389
           DELL    115.33000  116.09000  114.16000  115.24000  3874700
...                      ...        ...        ...        ...      ...
1996-01-03 WSM       2.38000    2.38000    2.22000    2.30000  1070400
1996-01-02 DECK      0.31944    0.31944    0.29166    0.31944   115200
           ERIE     19.33000   19.92000   19.33000   19.33000    24600
           TPL       1.74000    1.74000    1.72000    1.73000    55500
           WSM       2.30000    2.38000    2.30000    2.38000   886400

[70751 rows x 5 columns]

In [37]:
# Get unique tickers from combined_df2
unique_tickers = combined_df2.index.get_level_values('symbol').unique()

print(len(unique_tickers))

18


### Factor Definitions & Calculations:
1. **Quality:**
   - `net_profit_to_total_operate_revenue_ttm`: Measures profitability relative to revenue.
   - `DECM`: Detection of Earnings Manipulation, signals potential earnings manipulation.
   - `roe_ttm`: Return on Equity, indicates profitability against equity.
   - `GMI`: Gross Margin Index, measures changes in gross margin.
   - `ACCA`: Accruals, indicating earnings quality.

2. **Value:**
   - `financial_liability`: Total liabilities as a measure of financial leverage.
   - `cash_flow_to_price_ratio`: Valuation metric using cash flow.
   - `market_cap`: Total market value of the company's outstanding shares.
   - `net_profit_ttm`: Total net profit over the trailing twelve months.
   - `EBIT`: Earnings before interest and taxes, a measure of profitability.

3. **Emotional:**
   - `VOL20`: 20-day price volatility.
   - `DAVOL20`: 20-day average trading volume.
   - `VOSC`: Volume Oscillator, indicating market sentiment.
   - `VMACD`: Volume MACD, momentum indicator using volume.
   - `ATR14`: 14-day Average True Range, measures volatility.

4. **Growth:**
   - `PEG`: Price/Earnings to Growth ratio.
   - `net_profit_growth_rate`: Year-over-year growth in net profit.
   - `operating_revenue_growth_rate`: Year-over-year growth in revenue.
   - `net_asset_growth_rate`: Growth in net assets.
   - `net_operate_cashflow_growth_rate`: Growth in operating cash flow.

5. **Risk:**
   - `Variance20`: 20-day return variance.
   - `sharpe_ratio_20`: 20-day Sharpe ratio.
   - `Kurtosis20`: 20-day kurtosis of returns.
   - `Skewness20`: 20-day skewness of returns.
   - `sharpe_ratio_60`: 60-day Sharpe ratio.

6. **Stock:**
   - `net_asset_per_share`: Net assets per share.
   - `net_operate_cash_flow_per_share`: Operating cash flow per share.
   - `eps_ttm`: Earnings per share over trailing twelve months.
   - `retained_earnings_per_share`: Retained earnings per share.
   - `cashflow_per_share_ttm`: Cash flow per share over trailing twelve months.

7. **Momentum:**
   - `ROC20`: 20-day rate of change.
   - `Volume1M`: One-month trading volume.
   - `TRIX10`: 10-day triple exponential moving average.
   - `Price1M`: One-month price change.
   - `PLRC12`: 12-day price momentum.

8. **Technical:**
   - `MAC20`: 20-day moving average convergence divergence.
   - `boll_down`: Bollinger Band lower band.
   - `boll_up`: Bollinger Band upper band.
   - `MFI14`: 14-day Money Flow Index.

9. **Style:**
   - `size`: Size of the company (market cap).
   - `beta`: Beta value of the stock.
   - `momentum`: Momentum factor of the stock.
   - `book_to_price_ratio`: Book-to-price ratio.
   - `liquidity`: Liquidity of the stock.
   - `growth`: Growth factor of the stock.


In [40]:
# Define the list of factors to be fetched or calculated
factors = {
    'net_profit_to_total_revenue_ttm': 'Quality',
    #'DECM': 'Quality',
    'roe_ttm': 'Quality',
    'roa_ttm' : 'Quality',
    'GMI': 'Quality',
    'ACCA': 'Quality',
    'debt_to_asset_ratio' : 'Quality',

    'financial_liability': 'Value',
    'cash_flow_to_price_ratio_ttm': 'Value',
    'price_to_book_ratio' : 'Value',
    'price_to_sales_ratio_ttm' : 'Value',
    'price_to_earnings_ratio_ttm' : 'Value',
    'total_liability_to_total_asset_ratio' : 'Value',
    'net_profit_ttm': 'Value',
    #'EBIT_ttm': 'Value',
    'working_capital_ratio' : 'Value',
    'quick_ratio' : 'Value',
    'debt_to_equity_ratio' : 'Value',
    'operate_cash_flow_to_total_asset_ratio' : 'Value',
    'operate_cash_flow_to_total_liabilities_ratio' : 'Value',
    'operate_cash_flow_to_net_profit_ratio' : 'Value',
    'EV_to_operate_cash_flow_ratio' : 'Value',
    'debt_to_EBITDA_ratio' : 'Value',

    'EPS_growth_rate_ttm' : 'Growth',
    'PEG_ttm': 'Growth',
    'net_profit_growth_rate_ttm': 'Growth',
    'revenue_growth_rate_ttm': 'Growth',
    'net_asset_growth_rate': 'Growth',
    'operate_cash_flow_growth_rate_ttm': 'Growth',
    
    'net_asset_per_share': 'Stock',
    'net_operate_cash_flow_per_share_ttm': 'Stock',
    'retained_earnings_per_share': 'Stock',
    'market_cap(size)': 'Stock',   

    'VOL60': 'Emotional',
    'DAVOL60': 'Emotional',
    'VOSC': 'Emotional',
    'VMACD': 'Emotional',
    'ATR42': 'Emotional',

    'ROC60': 'Momentum',
    'Volume1Q': 'Momentum',
    'TRIX30': 'Momentum',
    'Price1Q': 'Momentum',
    'PLRC36': 'Momentum',

    'Variance60': 'Risk',
    'Skewness60': 'Risk',
    'Kurtosis60': 'Risk',
    'sharpe_ratio_20': 'Risk',
    'sharpe_ratio_60': 'Risk',

    'MACD60': 'Technical',
    'boll_down': 'Technical',
    'boll_up': 'Technical',
    'MFI42': 'Technical',

    'growth': 'Style',
    'momentum': 'Style',
    'beta': 'Style',
    'liquidity': 'Style',
}

# Create a list of factor names for iteration
factor_list = list(factors.keys())
print("Total number of factors:", len(factor_list))
print("Factor list:", factor_list)

Total number of factors: 54
Factor list: ['net_profit_to_total_revenue_ttm', 'roe_ttm', 'roa_ttm', 'GMI', 'ACCA', 'debt_to_asset_ratio', 'financial_liability', 'cash_flow_to_price_ratio_ttm', 'price_to_book_ratio', 'price_to_sales_ratio_ttm', 'price_to_earnings_ratio_ttm', 'total_liability_to_total_asset_ratio', 'net_profit_ttm', 'working_capital_ratio', 'quick_ratio', 'debt_to_equity_ratio', 'operate_cash_flow_to_total_asset_ratio', 'operate_cash_flow_to_total_liabilities_ratio', 'operate_cash_flow_to_net_profit_ratio', 'EV_to_operate_cash_flow_ratio', 'debt_to_EBITDA_ratio', 'EPS_growth_rate_ttm', 'PEG_ttm', 'net_profit_growth_rate_ttm', 'revenue_growth_rate_ttm', 'net_asset_growth_rate', 'operate_cash_flow_growth_rate_ttm', 'net_asset_per_share', 'net_operate_cash_flow_per_share_ttm', 'retained_earnings_per_share', 'market_cap(size)', 'VOL60', 'DAVOL60', 'VOSC', 'VMACD', 'ATR42', 'ROC60', 'Volume1Q', 'TRIX30', 'Price1Q', 'PLRC36', 'Variance60', 'Skewness60', 'Kurtosis60', 'sharpe_ra

#FACTORS FROM OHLCV DATA - DAILY

In [41]:
# Create a copy of combined_df and store the factors
combined_df3 = combined_df.reset_index()
combined_df3 = combined_df3.drop(combined_df3.columns[0], axis=1)
# Sort the DataFrame by date in ascending order (oldest to newest)
combined_df3['date'] = pd.to_datetime(combined_df3['date'])
combined_df3 = combined_df3.sort_values(['symbol', 'date'])

# Calculate risk-free rates
risk_free_rate_annual = 0.045
risk_free_rate_20 = (1 + risk_free_rate_annual) ** (20 / 252) - 1
risk_free_rate_60 = (1 + risk_free_rate_annual) ** (60 / 252) - 1

factor_columns = [
    'VOL60', 'DAVOL60', 'VOSC', 'VMACD', 'ATR42', 
    'ROC60', 'Volume1Q', 'TRIX30', 'Price1Q', 'PLRC36', 
    'Variance60', 'Skewness60', 'Kurtosis60', 'SharpeRatio20', 'SharpeRatio60', 
    'MACD60', 'boll_up', 'boll_down', 'MFI42',
    'GrowthRate', 'Momentum', 'Beta'
]

# Add all factor columns with NaN values
for col in factor_columns:
    combined_df3[col] = np.nan

#Emotional Factors
def calculate_volume_volatility_vol60(df, window=60):
    """Calculate 60-day volume volatility."""
    return df['close'].pct_change().rolling(window=window).std()

def calculate_volume_ma_davol60(df, window=60):
    """Calculate 60-day average volume."""
    return df['volume'].rolling(window=window).mean()

def calculate_volume_oscillator_vosc(df, window=60):
    """Calculate volume oscillator."""
    volume_ma = df['volume'].rolling(window=window).mean()
    return df['volume'] - volume_ma

def calculate_volume_macd(df, fast_span=36, slow_span=78):
    """Calculate volume MACD."""
    fast_ma = df['volume'].ewm(span=fast_span).mean()
    slow_ma = df['volume'].ewm(span=slow_span).mean()
    return fast_ma - slow_ma

def calculate_atr42(df, window=42):
    """Calculate 42-day Average True Range."""
    return (df['high'] - df['low']).rolling(window=window).mean()

#Momentum Factors
def calculate_rate_of_change_roc60(df, window=60):
    """Calculate 60-day rate of change."""
    return (df['close'] - df['close'].shift(window)) / df['close'].shift(window)

def calculate_volume_quarterly_1q(df, window=60):
    """Calculate quarterly volume."""
    return df['volume'].rolling(window=window).sum()

def calculate_trix30(df, span=30):
    """Calculate TRIX indicator."""
    triple_ema = df['close'].ewm(span=span).mean().ewm(span=span).mean().ewm(span=span).mean()
    return triple_ema.pct_change(periods=1)

def calculate_price_quarterly_price1q(df, window=60):
    """Calculate quarterly price change."""
    return df['close'] - df['close'].shift(window)

def calculate_price_level_ratio_PLRC36(df, window=36):
    """Calculate price level ratio."""
    return df['close'].rolling(window=window).mean() / df['close'].shift(window) - 1

#Risk Factors
def calculate_variance_60(df, window=60):
    """Calculate 60-day variance of returns."""
    return df['close'].pct_change().rolling(window=window).var()

def calculate_skewness60(df, window=60):
    """Calculate 60-day skewness of returns."""
    return df['close'].pct_change().rolling(window=window).skew()

def calculate_kurtosis60(df, window=60):
    """Calculate 60-day kurtosis of returns."""
    return df['close'].pct_change().rolling(window=window).kurt()

def calculate_sharpe_ratio(df, window, risk_free_rate):
    """Calculate Sharpe ratio for given window."""
    returns = df['close'].pct_change()
    annualized_returns = returns.rolling(window=window).mean() * 252
    annualized_std = returns.rolling(window=window).std() * np.sqrt(252)
    return (annualized_returns - risk_free_rate) / annualized_std

#Technical Factors
def calculate_macd_60(df, fast_span=36, slow_span=78):
    """Calculate MACD indicator."""
    fast_ma = df['close'].ewm(span=fast_span).mean()
    slow_ma = df['close'].ewm(span=slow_span).mean()
    return fast_ma - slow_ma

def calculate_bollinger_bands(df, window=60, num_std=2):
    """Calculate Bollinger Bands."""
    rolling_mean = df['close'].rolling(window=window).mean()
    rolling_std = df['close'].rolling(window=window).std()
    return {
        'boll_up': rolling_mean + (rolling_std * num_std),
        'boll_down': rolling_mean - (rolling_std * num_std)
    }

def calculate_mfi_42(df, window=42):
    """Calculate Money Flow Index."""
    typical_price = (df['close'] + df['high'] + df['low']) / 3
    raw_money_flow = typical_price * df['volume']
    positive_flow = raw_money_flow.where(typical_price > typical_price.shift(1), 0)
    negative_flow = raw_money_flow.where(typical_price < typical_price.shift(1), 0)
    positive_flow_sum = positive_flow.rolling(window=window).sum()
    negative_flow_sum = negative_flow.rolling(window=window).sum()
    money_flow_ratio = positive_flow_sum / negative_flow_sum
    return 100 - (100 / (1 + money_flow_ratio))

#Style Factors
def calculate_growth_rate(df, window=252):
    """Calculate growth rate using log returns."""
    return np.log(df['close'] / df['close'].shift(window))

def calculate_momentum(df, window=252):
    """Calculate momentum as price ratio."""
    return df['close'] / df['close'].shift(window)


# Get unique stock symbols
symbols = combined_df3['symbol'].unique()

# Process each stock separately
for symbol in symbols:
    try:
        # Get data for current symbol
        stock_data = combined_df3[combined_df3['symbol'] == symbol].copy()
        
        # Sort data in ascending order (oldest to newest) for calculations
        stock_data = stock_data.sort_values('date', ascending=True)
        
        # Calculate returns first
        stock_data['returns'] = stock_data['close'].pct_change()       
        
        # Emotional Factors
        stock_data['VOL60'] = calculate_volume_volatility_vol60(stock_data)
        stock_data['DAVOL60'] = calculate_volume_ma_davol60(stock_data)
        stock_data['VOSC'] = calculate_volume_oscillator_vosc(stock_data)
        stock_data['VMACD'] = calculate_volume_macd(stock_data)
        stock_data['ATR42'] = calculate_atr42(stock_data)

        # Momentum Factors
        stock_data['ROC60'] = calculate_rate_of_change_roc60(stock_data)
        stock_data['Volume1Q'] = calculate_volume_quarterly_1q(stock_data)
        stock_data['TRIX30'] = calculate_trix30(stock_data)
        stock_data['Price1Q'] = calculate_price_quarterly_price1q(stock_data)
        stock_data['PLRC36'] = calculate_price_level_ratio_PLRC36(stock_data)
        
        # Risk Factors
        stock_data['Variance60'] = calculate_variance_60(stock_data)
        stock_data['Skewness60'] = calculate_skewness60(stock_data)
        stock_data['Kurtosis60'] = calculate_kurtosis60(stock_data)
        stock_data['SharpeRatio20'] = calculate_sharpe_ratio(stock_data, 20, risk_free_rate_20)
        stock_data['SharpeRatio60'] = calculate_sharpe_ratio(stock_data, 60, risk_free_rate_60)
        
        # Technical Factors
        stock_data['MACD60'] = calculate_macd_60(stock_data)
        
        bollinger_bands = calculate_bollinger_bands(stock_data)
        stock_data['boll_up'] = bollinger_bands['boll_up']
        stock_data['boll_down'] = bollinger_bands['boll_down']
        
        stock_data['MFI42'] = calculate_mfi_42(stock_data) 

        # Style Factors
        stock_data['GrowthRate'] = calculate_growth_rate(stock_data)
        stock_data['Momentum'] = calculate_momentum(stock_data)
        
        # Forward fill any remaining NaN values (up to 5 days)
        for col in factor_columns:
            stock_data[col] = stock_data[col].ffill(limit=5)
        
        # Sort back to descending order (newest to oldest) before updating
        stock_data = stock_data.sort_values('date', ascending=False)
        
        # Update the original DataFrame with calculated values
        combined_df3.loc[combined_df3['symbol'] == symbol, factor_columns] = stock_data[factor_columns]
        
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        continue

# Sort back to descending order (newest to oldest) if needed
combined_df3 = combined_df3.sort_values(['symbol', 'date'], ascending=[True, False])



##Calculate Beta
# Sort combined_df3 from old date to new date
combined_df3 = combined_df3.sort_values(['symbol', 'date'], ascending=[True, True])

# Calculate stock returns for all stocks
combined_df3['stock_returns'] = combined_df3.groupby('symbol')['close'].pct_change()

# Get benchmark data and calculate returns
benchmark = '^GSPC'  # S&P 500 ETF benchmark

def get_benchmark_data():
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{benchmark}?serietype=line&apikey={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data and "historical" in data:
            return [(entry["date"], entry["close"]) for entry in data["historical"] if start_date <= entry["date"] <= end_date]
    except Exception as e:
        print(f"Error fetching benchmark data: {str(e)}")
    return []

# Fetch benchmark data
benchmark_data = get_benchmark_data()
benchmark_df = pd.DataFrame(benchmark_data, columns=['date', 'close'])
benchmark_df['date'] = pd.to_datetime(benchmark_df['date'])
benchmark_df.set_index('date', inplace=True)
benchmark_df.sort_index(ascending=True, inplace=True)

# Calculate benchmark returns
benchmark_df['market_returns'] = benchmark_df['close'].pct_change()

# Merge benchmark returns with combined_df3
#drop any existing market_returns column if it exists
if 'market_returns' in combined_df3.columns:
    combined_df3 = combined_df3.drop('market_returns', axis=1)

combined_df3 = combined_df3.merge(
    benchmark_df['market_returns'].reset_index(),
    on='date',
    how='left'
)

# Calculate beta for all dates
window = 60  # 60-day rolling window

# Group by symbol and calculate rolling beta
for symbol in combined_df3['symbol'].unique():
    try:
        # Get data for current symbol
        symbol_data = combined_df3[combined_df3['symbol'] == symbol].copy()
        
        # Calculate rolling beta
        rolling_beta = (symbol_data['stock_returns'].rolling(window=window, min_periods=30)
                       .cov(symbol_data['market_returns']) / 
                       symbol_data['market_returns'].rolling(window=window, min_periods=30).var())
        
        # Update beta values in combined_df3
        combined_df3.loc[combined_df3['symbol'] == symbol, 'Beta'] = rolling_beta.values
        
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        continue

# Forward fill any remaining NaN values (up to 5 days)
combined_df3['Beta'] = combined_df3.groupby('symbol')['Beta'].ffill(limit=5)

# Sort back to descending order (newest to oldest)
combined_df3 = combined_df3.sort_values(['symbol', 'date'], ascending=[True, False]).reset_index(drop=True)

In [43]:
print("\nFirst few rows of the combined DataFrame with factors:")
display(combined_df3.round(4))

print("\nNumber of NaN Values for each factor:")
print(combined_df3[factor_columns].isna().sum())


First few rows of the combined DataFrame with factors:


,date,symbol,open,high,low,close,volume,VOL60,DAVOL60,VOSC,...,SharpeRatio60,MACD60,boll_up,boll_down,MFI42,GrowthRate,Momentum,Beta,stock_returns,market_returns
0,2024-12-31,APO,167.00,167.60,164.93,165.16,2817031,0.0222,5.337859e+06,-2.520828e+06,...,2.5144,12.4369,188.8871,131.3433,74.1610,0.5723,1.7723,1.9742,-0.0081,-0.0043
1,2024-12-30,APO,167.95,168.87,165.18,166.51,2597815,0.0224,5.355032e+06,-2.757217e+06,...,2.9429,12.8246,189.0584,130.1776,75.1014,0.5689,1.7663,1.9923,-0.0221,-0.0107
2,2024-12-27,APO,172.68,173.41,169.70,170.28,2859065,0.0223,5.371598e+06,-2.512533e+06,...,3.1108,13.1840,189.3019,128.7624,75.9832,0.6031,1.8278,1.9871,-0.0235,-0.0111
3,2024-12-26,APO,173.63,175.08,172.51,174.37,1348439,0.0220,5.424011e+06,-4.075572e+06,...,3.4993,13.4366,189.2781,127.5336,76.9292,0.6330,1.8832,1.9719,-0.0027,-0.0004
4,2024-12-24,APO,172.51,174.85,171.80,174.85,1023900,0.0229,5.496602e+06,-4.472702e+06,...,4.0381,13.5633,189.0212,126.3785,76.2548,0.6467,1.9093,1.7709,0.0188,0.0110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,1996-01-08,WSM,2.27,2.28,2.20,2.28,292800,NaN,NaN,NaN,...,NaN,-0.0013,NaN,NaN,NaN,NaN,NaN,NaN,0.0044,0.0028
70747,1996-01-05,WSM,2.27,2.27,2.20,2.27,2770400,NaN,NaN,NaN,...,NaN,-0.0013,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,-0.0016
70748,1996-01-04,WSM,2.30,2.30,2.20,2.27,1379200,NaN,NaN,NaN,...,NaN,-0.0011,NaN,NaN,NaN,NaN,NaN,NaN,-0.0130,-0.0058
70749,1996-01-03,WSM,2.38,2.38,2.22,2.30,1070400,NaN,NaN,NaN,...,NaN,-0.0006,NaN,NaN,NaN,NaN,NaN,NaN,-0.0336,0.0010



Number of NaN Values for each factor:
VOL60            1080
DAVOL60          1062
VOSC             1062
VMACD               0
ATR42             738
ROC60            1080
Volume1Q         1062
TRIX30             18
Price1Q          1080
PLRC36            648
Variance60       1080
Skewness60       1080
Kurtosis60       1080
SharpeRatio20     360
SharpeRatio60    1080
MACD60              0
boll_up          1062
boll_down        1062
MFI42             738
GrowthRate       4536
Momentum         4536
Beta              540
dtype: int64


In [46]:
# Function to display NaN values for each stock
def display_stock_nan_values(df):
    """
    Display total NaN values for each stock.
    
    Args:
        df: DataFrame containing the factors
    """
    # Get all factor columns
    factor_columns = [
        'GrowthRate', 'Momentum', 'Beta',
        'VOL60', 'DAVOL60', 'VOSC', 'VMACD', 'ATR42',
        'Variance60', 'Skewness60', 'Kurtosis60', 'SharpeRatio20', 'SharpeRatio60',
        'ROC60', 'Volume1Q', 'TRIX30', 'Price1Q', 'PLRC36',
        'MACD60', 'boll_up', 'boll_down', 'MFI42'
    ]
    
    # Group by symbol and count NaN values
    nan_counts = df.groupby('symbol')[factor_columns].apply(lambda x: x.isna().sum())
    
    # Calculate total NaN values for each stock
    nan_counts['Total_NaN'] = nan_counts.sum(axis=1)
    
    # Sort by total NaN values in descending order
    nan_counts = nan_counts.sort_values('Total_NaN', ascending=False)
    
    # Display results
    print("\nNaN values for each stock:")
    display(nan_counts)

# Call the function
display_stock_nan_values(combined_df3)


NaN values for each stock:


,GrowthRate,Momentum,Beta,VOL60,DAVOL60,VOSC,VMACD,ATR42,Variance60,Skewness60,...,ROC60,Volume1Q,TRIX30,Price1Q,PLRC36,MACD60,boll_up,boll_down,MFI42,Total_NaN
symbol,,,,,,,,,,,,,,,,,,,,,
APO,252,252,30,60,59,59,0,41,60,60,...,60,59,1,60,36,0,59,59,41,1388
CRWD,252,252,30,60,59,59,0,41,60,60,...,60,59,1,60,36,0,59,59,41,1388
WDAY,252,252,30,60,59,59,0,41,60,60,...,60,59,1,60,36,0,59,59,41,1388
VST,252,252,30,60,59,59,0,41,60,60,...,60,59,1,60,36,0,59,59,41,1388
TPL,252,252,30,60,59,59,0,41,60,60,...,60,59,1,60,36,0,59,59,41,1388
TKO,252,252,30,60,59,59,0,41,60,60,...,60,59,1,60,36,0,59,59,41,1388
SW,252,252,30,60,59,59,0,41,60,60,...,60,59,1,60,36,0,59,59,41,1388
SMCI,252,252,30,60,59,59,0,41,60,60,...,60,59,1,60,36,0,59,59,41,1388
PLTR,252,252,30,60,59,59,0,41,60,60,...,60,59,1,60,36,0,59,59,41,1388


In [52]:
daily_tech_factors = combined_df3[['symbol', 'date'] + factor_columns].reset_index(drop=True)
display(daily_tech_factors.round(4))

#daily_tech_factors.to_csv('daily_tech_factors.csv', index = False)

,symbol,date,VOL60,DAVOL60,VOSC,VMACD,ATR42,ROC60,Volume1Q,TRIX30,...,Kurtosis60,SharpeRatio20,SharpeRatio60,MACD60,boll_up,boll_down,MFI42,GrowthRate,Momentum,Beta
0,APO,2024-12-31,0.0222,5.337859e+06,-2.520828e+06,1.857544e+06,4.5240,0.2204,320271517.0,0.0052,...,5.4568,-1.7094,2.5144,12.4369,188.8871,131.3433,74.1610,0.5723,1.7723,1.9742
1,APO,2024-12-30,0.0224,5.355032e+06,-2.757217e+06,2.057539e+06,4.4969,0.2676,321301941.0,0.0053,...,5.0174,-2.0017,2.9429,12.8246,189.0584,130.1776,75.1014,0.5689,1.7663,1.9923
2,APO,2024-12-27,0.0223,5.371598e+06,-2.512533e+06,2.278233e+06,4.4710,0.2832,322295854.0,0.0054,...,5.2109,-0.7858,3.1108,13.1840,189.3019,128.7624,75.9832,0.6031,1.8278,1.9871
3,APO,2024-12-26,0.0220,5.424011e+06,-4.075572e+06,2.506073e+06,4.4281,0.3209,325440650.0,0.0056,...,5.5075,-0.2100,3.4993,13.4366,189.2781,127.5336,76.9292,0.6330,1.8832,1.9719
4,APO,2024-12-24,0.0229,5.496602e+06,-4.472702e+06,2.796554e+06,4.4386,0.3998,329796111.0,0.0057,...,4.3796,0.1601,4.0381,13.5633,189.0212,126.3785,76.2548,0.6467,1.9093,1.7709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70746,WSM,1996-01-08,NaN,NaN,NaN,2.009399e+03,NaN,NaN,NaN,-0.0027,...,NaN,NaN,NaN,-0.0013,NaN,NaN,NaN,NaN,NaN,NaN
70747,WSM,1996-01-05,NaN,NaN,NaN,2.263433e+04,NaN,NaN,NaN,-0.0033,...,NaN,NaN,NaN,-0.0013,NaN,NaN,NaN,NaN,NaN,NaN
70748,WSM,1996-01-04,NaN,NaN,NaN,4.928680e+03,NaN,NaN,NaN,-0.0039,...,NaN,NaN,NaN,-0.0011,NaN,NaN,NaN,NaN,NaN,NaN
70749,WSM,1996-01-03,NaN,NaN,NaN,1.376068e+03,NaN,NaN,NaN,-0.0046,...,NaN,NaN,NaN,-0.0006,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Convert date to datetime if not already
daily_tech_factors['date'] = pd.to_datetime(daily_tech_factors['date'])

quarterly_dates = pd.date_range(start=start_date, end=end_date, freq='QE').strftime("%Y-%m-%d").tolist()
quarterly_dates = pd.to_datetime(quarterly_dates)

# Resample daily data to quarterly
quarterly_tech_factors = daily_tech_factors.copy()
quarterly_tech_factors['date'] = pd.to_datetime(quarterly_tech_factors['date'])

# Group by symbol and resample to quarterly frequency
quarterly_tech_factors = (quarterly_tech_factors
    .set_index(['symbol', 'date'])
    .groupby('symbol')
    .resample('QE', level='date')
    .last()
    .reset_index()
)

# Filter for only the dates in our quarterly_dates list
quarterly_tech_factors = quarterly_tech_factors[
    quarterly_tech_factors['date'].isin(quarterly_dates)
]

# Sort by symbol and date in descending order (latest first)
quarterly_tech_factors = quarterly_tech_factors.sort_values(['symbol', 'date'], ascending=[True, False])

# Drop the oldest quarter for each stock
quarterly_tech_factors = quarterly_tech_factors.groupby('symbol').apply(
    lambda x: x.iloc[:-1]).reset_index(drop=True)

C:\Users\Koushik Hati\AppData\Local\Temp\ipykernel_16984\1870851044.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  quarterly_tech_factors = quarterly_tech_factors.groupby('symbol').apply(


In [55]:
# Display results
print("\nDate range in quarterly technical factors:")
print("Earliest date:", quarterly_tech_factors['date'].min())
print("Latest date:", quarterly_tech_factors['date'].max())

print("\nFirst few rows of quarterly technical factors:")
display(quarterly_tech_factors.head().round(4))

# Check for any missing values
print("\nMissing values in quarterly technical factors:")
display(quarterly_tech_factors.isna().sum())

#Save the Quarterly Tech Factors File
print("\nQuarterly Technical Factors are saved to 'quarterly_tech_factors.csv'")
quarterly_tech_factors.to_csv('quarterly_tech_factors.csv', index = False)


Date range in quarterly technical factors:
Earliest date: 1996-06-30 00:00:00
Latest date: 2024-12-31 00:00:00

First few rows of quarterly technical factors:


,symbol,date,VOL60,DAVOL60,VOSC,VMACD,ATR42,ROC60,Volume1Q,TRIX30,...,Kurtosis60,SharpeRatio20,SharpeRatio60,MACD60,boll_up,boll_down,MFI42,GrowthRate,Momentum,Beta
0,APO,2024-12-31,0.0222,5.337859e+06,-2.520828e+06,1.857544e+06,4.5240,0.2204,320271517.0,0.0052,...,5.4568,-1.7094,2.5144,12.4369,188.8871,131.3433,74.1610,0.5723,1.7723,1.9742
1,APO,2024-09-30,0.0247,2.823938e+06,8.614266e+05,6.861282e+04,3.1517,0.0520,169436303.0,0.0000,...,6.3966,3.5566,0.7142,1.8385,130.2349,100.1208,48.2907,0.3156,1.3711,1.6482
2,APO,2024-06-30,0.0188,2.439068e+06,1.256403e+07,3.953631e+05,2.9414,0.0487,146344065.0,0.0011,...,0.4483,0.6094,0.7799,3.0808,120.0960,107.0044,59.8984,0.4519,1.5713,1.2153
3,APO,2024-03-31,0.0131,2.465437e+06,-9.500371e+05,-7.416909e+04,2.4193,0.2312,147926226.0,0.0030,...,-0.3481,0.4782,4.2635,5.5371,119.3683,92.1620,55.9307,0.6433,1.9027,0.8816
4,APO,2023-12-31,0.0179,2.519908e+06,-1.901764e+05,-8.213699e+04,2.0233,0.0593,151194505.0,0.0014,...,5.9976,0.8416,0.9542,2.6887,96.7235,79.8855,67.2045,0.3992,1.4906,1.1616



Missing values in quarterly technical factors:


symbol            0
date              0
VOL60             0
DAVOL60           0
VOSC              0
VMACD             0
ATR42             0
ROC60             0
Volume1Q          0
TRIX30            0
Price1Q           0
PLRC36            0
Variance60        0
Skewness60        0
Kurtosis60        0
SharpeRatio20     0
SharpeRatio60     0
MACD60            0
boll_up           0
boll_down         0
MFI42             0
GrowthRate       51
Momentum         51
Beta              0
dtype: int64


Quarterly Technical Factors are saved to 'quarterly_tech_factors.csv'


In [56]:
quarterly_tech_factors

,symbol,date,VOL60,DAVOL60,VOSC,VMACD,ATR42,ROC60,Volume1Q,TRIX30,...,Kurtosis60,SharpeRatio20,SharpeRatio60,MACD60,boll_up,boll_down,MFI42,GrowthRate,Momentum,Beta
0,APO,2024-12-31,0.022233,5.337859e+06,-2.520828e+06,1.857544e+06,4.524048,0.220424,320271517.0,0.005159,...,5.456828,-1.709388,2.514436,12.436933,188.887070,131.343263,74.160994,0.572274,1.772293,1.974168
1,APO,2024-09-30,0.024733,2.823938e+06,8.614266e+05,6.861282e+04,3.151667,0.051962,169436303.0,0.000047,...,6.396609,3.556553,0.714179,1.838533,130.234858,100.120809,48.290705,0.315636,1.371131,1.648160
2,APO,2024-06-30,0.018779,2.439068e+06,1.256403e+07,3.953631e+05,2.941429,0.048672,146344065.0,0.001120,...,0.448297,0.609433,0.779916,3.080801,120.095969,107.004364,59.898434,0.451925,1.571334,1.215277
3,APO,2024-03-31,0.013089,2.465437e+06,-9.500371e+05,-7.416909e+04,2.419286,0.231249,147926226.0,0.003024,...,-0.348075,0.478176,4.263547,5.537078,119.368341,92.161992,55.930718,0.643278,1.902707,0.881649
4,APO,2023-12-31,0.017887,2.519908e+06,-1.901764e+05,-8.213699e+04,2.023333,0.059338,151194505.0,0.001391,...,5.997634,0.841613,0.954173,2.688703,96.723526,79.885474,67.204454,0.399154,1.490563,1.161575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,WSM,1997-06-30,0.029035,2.291693e+06,2.686707e+06,-1.384261e+05,0.162143,0.517045,137501600.0,0.004054,...,0.614245,3.987460,4.012135,0.281617,5.149599,3.440735,75.363091,0.534193,1.706070,0.435565
1114,WSM,1997-03-31,0.033290,3.286360e+06,5.490440e+06,6.115990e+05,0.167857,-0.172811,197181600.0,-0.001798,...,0.017029,3.330999,-1.264485,-0.033041,4.628652,3.060348,48.554462,0.234348,1.264085,0.573329
1115,WSM,1996-12-31,0.035281,1.996120e+06,6.734800e+05,-1.722852e+05,0.182857,0.178756,119767200.0,0.002550,...,-0.066218,1.330667,1.488558,0.197118,4.490426,3.313908,65.411915,0.682218,1.978261,0.533917
1116,WSM,1996-09-30,0.053099,2.742067e+06,3.114733e+06,5.313229e+05,0.162857,0.254417,164524000.0,0.003434,...,2.811217,1.974694,1.515121,0.170055,3.850566,2.165101,75.049236,NaN,NaN,3.615154


#FACTORS FROM FINANCIAL DATA - QUARTERLY

In [57]:
#Fetching Financial Statements
# Load final_tickers
with open("final_tickers.pickle", "rb") as f:
    final_tickers = pickle.load(f)

# Load combined_df2 to determine the date range (if needed)
# Assuming combined_df2 already exists
start_date = combined_df2.index.get_level_values('date').min().strftime('%Y-%m-%d')
end_date = combined_df2.index.get_level_values('date').max().strftime('%Y-%m-%d')
print(f"Fetching financials from {start_date} to {end_date}")

# Create directory to store individual financial statements as CSV
os.makedirs("financials_csv", exist_ok=True)

# Function to fetch financial data for a symbol
def fetch_financial_data(symbol, statement_type):
    url = f"{base_url}/{statement_type}/{symbol}"
    params = {
        "period": "quarter",
        "apikey": api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.DataFrame(response.json())
        if not df.empty:
            df['symbol'] = symbol
            df['statement_type'] = statement_type
        return df
    else:
        print(f"Failed to fetch {statement_type} for {symbol} | Status Code: {response.status_code}")
        return pd.DataFrame()

# Main loop to fetch all financials and save to CSV
financial_data = {}

for i, symbol in enumerate(final_tickers):
    print(f"\n[{i+1}/{len(final_tickers)}] Fetching financials for {symbol}...")

    income_df = fetch_financial_data(symbol, "income-statement")
    balance_df = fetch_financial_data(symbol, "balance-sheet-statement")
    cashflow_df = fetch_financial_data(symbol, "cash-flow-statement")

    # Optional: Filter by date range if needed
    for merged_df2_with_ttm in [income_df, balance_df, cashflow_df]:
        if not merged_df2_with_ttm.empty and 'date' in merged_df2_with_ttm.columns:
            merged_df2_with_ttm['date'] = pd.to_datetime(merged_df2_with_ttm['date'])
            merged_df2_with_ttm.dropna(subset=['date'], inplace=True)

    income_df = income_df[income_df['date'].between(start_date, end_date)]
    balance_df = balance_df[balance_df['date'].between(start_date, end_date)]
    cashflow_df = cashflow_df[cashflow_df['date'].between(start_date, end_date)]

    # Save CSVs
    if not income_df.empty:
        income_df.to_csv(f"financials_csv/{symbol}_income_statement.csv", index=False)
    if not balance_df.empty:
        balance_df.to_csv(f"financials_csv/{symbol}_balance_sheet.csv", index=False)
    if not cashflow_df.empty:
        cashflow_df.to_csv(f"financials_csv/{symbol}_cash_flow_statement.csv", index=False)

    # Save to dictionary
    financial_data[symbol] = {
        "income-statement": income_df,
        "balance-sheet-statement": balance_df,
        "cash-flow-statement": cashflow_df
    }

    time.sleep(0.5)  # API rate limit

# # Optionally, save the financial data to a pickle file as well
# with open("financial_statements_quarterly.pickle", "wb") as f:
#     pickle.dump(financial_data, f)

#print("\nAll financial statements saved to 'financials_csv' and 'financial_statements_quarterly.pickle'.")
print("\nAll financial statements saved to 'financials_csv'.")

Fetching financials from 1996-01-02 to 2024-12-31

[1/18] Fetching financials for DASH...

[2/18] Fetching financials for EXE...

[3/18] Fetching financials for TKO...

[4/18] Fetching financials for WSM...

[5/18] Fetching financials for APO...

[6/18] Fetching financials for LII...

[7/18] Fetching financials for WDAY...

[8/18] Fetching financials for TPL...

[9/18] Fetching financials for DELL...

[10/18] Fetching financials for ERIE...

[11/18] Fetching financials for PLTR...

[12/18] Fetching financials for SW...

[13/18] Fetching financials for CRWD...

[14/18] Fetching financials for GDDY...

[15/18] Fetching financials for KKR...

[16/18] Fetching financials for VST...

[17/18] Fetching financials for DECK...

[18/18] Fetching financials for SMCI...

All financial statements saved to 'financials_csv'.


In [59]:
def print_financial_statement(symbol, statement_type, start_date=None, end_date=None):
    """
    Print financial statement for a given stock and period.
    
    Args:
        symbol (str): Stock symbol
        statement_type (str): Type of statement ('income', 'balance', or 'cashflow')
        start_date (str): Start date in 'YYYY-MM-DD' format (optional)
        end_date (str): End date in 'YYYY-MM-DD' format (optional)
    """
    # Validate statement type
    valid_statements = {
        "income": "income_statement",
        "balance": "balance_sheet",
        "cashflow": "cash_flow_statement"
    }
    
    if statement_type.lower() not in valid_statements:
        print(f"❌ Invalid statement type. Please choose from: {', '.join(valid_statements.keys())}")
        return
    
    # Build file path
    file_path = f"financials_csv/{symbol}_{valid_statements[statement_type.lower()]}.csv"
    
    try:
        # Load the CSV
        df = pd.read_csv(file_path, parse_dates=["date"])
        
        # Print basic information
        print(f"\n📊 {statement_type.upper()} Statement for {symbol}")
        print(f"📅 Date Range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
        print(f"📄 Total number of periods: {len(df)}")
        
        # Filter by date if provided
        if start_date or end_date:
            if start_date:
                df = df[df["date"] >= pd.to_datetime(start_date)]
            if end_date:
                df = df[df["date"] <= pd.to_datetime(end_date)]
            print(f"📅 Filtered period: {start_date or 'beginning'} to {end_date or 'end'}")
        
        # Print the data
        #print(f"\n�� Statement data:")
        display(df.sort_values('date', ascending=False))

        print(f"\n📋 Available columns:")
        print(df.columns.tolist())
        
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        print("Please check if the stock symbol is correct and the file exists.")
    
    except Exception as e:
        print(f"⚠️ Error: {e}")


# Example usage:
print_financial_statement("APO", "income", "2023-01-01", "2024-12-31")    


📊 INCOME Statement for APO
📅 Date Range: 2007-03-31 to 2024-12-31
📄 Total number of periods: 67
📅 Filtered period: 2023-01-01 to 2024-12-31


,date,symbol,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,revenue,costOfRevenue,...,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,link,finalLink,statement_type
0,2024-12-31,APO,USD,1858681,2025-02-24,2025-02-24 16:40:05,2024,Q4,5283000000,289000000,...,62000000,1486000000,0.281280,2.42,2.39,584000000,603000000,https://www.sec.gov/Archives/edgar/data/185868...,https://www.sec.gov/Archives/edgar/data/185868...,income-statement
1,2024-09-30,APO,USD,1858681,2024-11-06,2024-11-06 17:02:40,2024,Q3,7773000000,1275000000,...,317000000,811000000,0.104336,4.10,4.08,585400000,589000000,https://www.sec.gov/Archives/edgar/data/185868...,https://www.sec.gov/Archives/edgar/data/185868...,income-statement
2,2024-06-30,APO,USD,1858681,2024-08-08,2024-08-07 18:17:49,2024,Q2,6018000000,291000000,...,261000000,853000000,0.141741,1.36,1.35,587100000,590200000,https://www.sec.gov/Archives/edgar/data/185868...,https://www.sec.gov/Archives/edgar/data/185868...,income-statement
3,2024-03-31,APO,USD,1858681,2024-05-07,2024-05-07 16:03:22,2024,Q1,7040000000,269000000,...,422000000,1427000000,0.202699,2.31,2.28,588120328,605378709,https://www.sec.gov/Archives/edgar/data/185868...,https://www.sec.gov/Archives/edgar/data/185868...,income-statement
4,2023-12-31,APO,USD,1858681,2024-02-27,2024-02-27 16:08:55,2023,Q4,11046000000,261000000,...,-1620000000,2756000000,0.249502,4.75,4.65,584000000,601000000,https://www.sec.gov/Archives/edgar/data/185868...,https://www.sec.gov/Archives/edgar/data/185868...,income-statement
5,2023-09-30,APO,USD,1858681,2023-11-07,2023-11-07 16:29:58,2023,Q3,2595000000,254000000,...,243000000,682000000,0.262813,0.68,0.68,578797225,578797225,https://www.sec.gov/Archives/edgar/data/185868...,https://www.sec.gov/Archives/edgar/data/185868...,income-statement
6,2023-06-30,APO,USD,1858681,2023-08-07,2023-08-07 16:02:51,2023,Q2,13702000000,257000000,...,201000000,599000000,0.043716,1.00,1.00,579000000,579000000,https://www.sec.gov/Archives/edgar/data/185868...,https://www.sec.gov/Archives/edgar/data/185868...,income-statement
7,2023-03-31,APO,USD,1858681,2023-05-09,2023-05-09 16:18:23,2023,Q1,5301000000,255000000,...,253000000,1010000000,0.190530,1.27,1.27,584115927,584242571,https://www.sec.gov/Archives/edgar/data/185868...,https://www.sec.gov/Archives/edgar/data/185868...,income-statement



📋 Available columns:
['date', 'symbol', 'reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'calendarYear', 'period', 'revenue', 'costOfRevenue', 'grossProfit', 'grossProfitRatio', 'researchAndDevelopmentExpenses', 'generalAndAdministrativeExpenses', 'sellingAndMarketingExpenses', 'sellingGeneralAndAdministrativeExpenses', 'otherExpenses', 'operatingExpenses', 'costAndExpenses', 'interestIncome', 'interestExpense', 'depreciationAndAmortization', 'ebitda', 'ebitdaratio', 'operatingIncome', 'operatingIncomeRatio', 'totalOtherIncomeExpensesNet', 'incomeBeforeTax', 'incomeBeforeTaxRatio', 'incomeTaxExpense', 'netIncome', 'netIncomeRatio', 'eps', 'epsdiluted', 'weightedAverageShsOut', 'weightedAverageShsOutDil', 'link', 'finalLink', 'statement_type']


In [61]:
def load_financial_statements(csv_dir="financials_csv"):
    """
    Load and combine financial statements for all stocks into three separate dataframes.
    
    Args:
        csv_dir (str): Directory containing financial statement CSV files
        
    Returns:
        tuple: (income_df_all, balance_df_all, cashflow_df_all) containing combined financial statements
    """
    # Predefine the selected columns
    balance_cols = ['date', 'symbol', 'reportedCurrency', 'cik', 'totalAssets', 'totalCurrentAssets', 
                    'totalLiabilities', 'totalCurrentLiabilities', 'cashAndCashEquivalents',
                    'inventory', 'totalDebt', 'retainedEarnings', 'totalStockholdersEquity']

    income_cols = ['date', 'symbol', 'reportedCurrency', 'cik', 'revenue', 'grossProfit', 'netIncome', 'eps',
                   'ebitda', 'weightedAverageShsOut']

    cashflow_cols = ['date', 'symbol', 'reportedCurrency', 'cik', 'operatingCashFlow']

    # Initialize empty lists to collect data
    income_dfs = []
    balance_dfs = []
    cashflow_dfs = []
    
    # Get list of all available stock symbols from the directory
    available_symbols = set()
    for file in os.listdir(csv_dir):
        if file.endswith('_income_statement.csv'):
            symbol = file.split('_')[0]
            available_symbols.add(symbol)
    
    print(f"Found {len(available_symbols)} stocks with financial statements")
    
    # Loop through all available tickers and read their statements
    for symbol in sorted(available_symbols):
        try:
            # Load income statement
            income_path = os.path.join(csv_dir, f"{symbol}_income_statement.csv")
            if os.path.exists(income_path):
                df_income = pd.read_csv(income_path, parse_dates=["date"])
                df_income = df_income[income_cols]
                income_dfs.append(df_income)

            # Load balance sheet
            balance_path = os.path.join(csv_dir, f"{symbol}_balance_sheet.csv")
            if os.path.exists(balance_path):
                df_balance = pd.read_csv(balance_path, parse_dates=["date"])
                df_balance = df_balance[balance_cols]
                balance_dfs.append(df_balance)

            # Load cash flow statement
            cashflow_path = os.path.join(csv_dir, f"{symbol}_cash_flow_statement.csv")
            if os.path.exists(cashflow_path):
                df_cashflow = pd.read_csv(cashflow_path, parse_dates=["date"])
                df_cashflow = df_cashflow[cashflow_cols]
                cashflow_dfs.append(df_cashflow)

        except Exception as e:
            print(f"⚠️ Error processing {symbol}: {e}")

    # Concatenate dataframes
    income_df_all = pd.concat(income_dfs, ignore_index=True)
    balance_df_all = pd.concat(balance_dfs, ignore_index=True)
    cashflow_df_all = pd.concat(cashflow_dfs, ignore_index=True)

    # Sort all dataframes by date and symbol
    for df in [income_df_all, balance_df_all, cashflow_df_all]:
        df.sort_values(['date', 'symbol'], ascending=[False, True], inplace=True)

    # Print summary information
    # print("\nCombined Financial Statements Summary:")
    # print(f"Income Statement: {income_df_all.shape[0]} rows, {income_df_all.shape[1]} columns")
    # print(f"Balance Sheet: {balance_df_all.shape[0]} rows, {balance_df_all.shape[1]} columns")
    # print(f"Cash Flow Statement: {cashflow_df_all.shape[0]} rows, {cashflow_df_all.shape[1]} columns")
    # print(f"Income DF shape: {income_df_all.shape}")
    # print(f"Balance DF shape: {balance_df_all.shape}")
    # print(f"Cashflow DF shape: {cashflow_df_all.shape}")  

    print("\nDate Range:")
    print(f"From: {min(income_df_all['date'].min(), balance_df_all['date'].min(), cashflow_df_all['date'].min())}")
    print(f"To: {max(income_df_all['date'].max(), balance_df_all['date'].max(), cashflow_df_all['date'].max())}")
    
    print("\nNumber of unique stocks in each statement:")
    print(f"Income Statement: {income_df_all['symbol'].nunique()}")
    print(f"Balance Sheet: {balance_df_all['symbol'].nunique()}")
    print(f"Cash Flow Statement: {cashflow_df_all['symbol'].nunique()}")

    return income_df_all, balance_df_all, cashflow_df_all

# Load the financial statements
income_df_all, balance_df_all, cashflow_df_all = load_financial_statements()

# Optional: Save the combined dataframes to CSV files
# income_df.to_csv('combined_income_statements.csv', index=False)
# balance_df.to_csv('combined_balance_sheets.csv', index=False)
# cashflow_df.to_csv('combined_cashflow_statements.csv', index=False)

Found 18 stocks with financial statements

Date Range:
From: 1996-01-28 00:00:00
To: 2024-12-31 00:00:00

Number of unique stocks in each statement:
Income Statement: 18
Balance Sheet: 18
Cash Flow Statement: 18


In [62]:
income_df_all.columns

Index(['date', 'symbol', 'reportedCurrency', 'cik', 'revenue', 'grossProfit',
       'netIncome', 'eps', 'ebitda', 'weightedAverageShsOut'],
      dtype='object')

In [63]:
# Merge the dataframes
merged_df = income_df_all.merge(balance_df_all, on=["date", "symbol", "reportedCurrency", "cik"], how="inner")  # Using inner join to ensure we only keep rows with data in all statements

merged_df = merged_df.merge(cashflow_df_all, on=["date", "symbol", "reportedCurrency", "cik"], how="inner")

# Sort the merged dataframe
merged_df = merged_df.sort_values(['date', 'symbol'], ascending=[False, True])

# Print information about the merged dataframe
print("Merged Financial Statements Summary:")
print(f"Total number of rows: {len(merged_df)}")
print(f"Total number of columns: {len(merged_df.columns)}")
print(f"Number of unique stocks: {merged_df['symbol'].nunique()}")
#print(f"Date range: {merged_df['date'].min()} to {merged_df['date'].max()}")

#Save to CSV
merged_df.to_csv('merged_financial_data.csv', index=False)

print("\nFirst few rows of merged dataframe:")
merged_df.head()

Merged Financial Statements Summary:
Total number of rows: 1150
Total number of columns: 20
Number of unique stocks: 18

First few rows of merged dataframe:


,date,symbol,reportedCurrency,cik,revenue,grossProfit,netIncome,eps,ebitda,weightedAverageShsOut,totalAssets,totalCurrentAssets,totalLiabilities,totalCurrentLiabilities,cashAndCashEquivalents,inventory,totalDebt,retainedEarnings,totalStockholdersEquity,operatingCashFlow
0,2024-12-31,APO,USD,1858681.0,5283000000,4994000000,1.486000e+09,2.42,2.112000e+09,584000000,3.778950e+11,2.157040e+11,3.469150e+11,2.696150e+11,1.616600e+10,0,10588000000,6.022000e+09,1.725300e+10,-4000000
1,2024-12-31,DASH,USD,1792789.0,2873000000,1420000000,1.410000e+08,0.34,2.580000e+08,417056000,1.284500e+10,7.386000e+09,5.035000e+09,4.438000e+09,4.019000e+09,0,536000000,-5.255000e+09,7.803000e+09,518000000
2,2024-12-31,DECK,USD,910521.0,1827165000,1102623000,4.567340e+08,3.01,6.017130e+08,151820000,3.964353e+09,3.274112e+09,1.333434e+09,1.032882e+09,2.240923e+09,576669000,257029000,2.424898e+09,2.630919e+09,1095415000
3,2024-12-31,ERIE,USD,922621.0,338709000,338575000,1.520290e+08,3.26,0.000000e+00,46191602,2.888614e+09,0.000000e+00,9.013560e+08,0.000000e+00,2.748380e+08,0,0,3.162303e+09,1.987258e+09,193458000
4,2024-12-31,EXE,USD,895126.0,1999000000,592000000,-3.990000e+08,-1.72,2.900000e+08,231539000,2.789400e+10,1.997000e+09,1.032900e+10,3.123000e+09,3.170000e+08,0,5751000000,3.876000e+09,1.756500e+10,382000000


In [64]:
merged_df.columns

Index(['date', 'symbol', 'reportedCurrency', 'cik', 'revenue', 'grossProfit',
       'netIncome', 'eps', 'ebitda', 'weightedAverageShsOut', 'totalAssets',
       'totalCurrentAssets', 'totalLiabilities', 'totalCurrentLiabilities',
       'cashAndCashEquivalents', 'inventory', 'totalDebt', 'retainedEarnings',
       'totalStockholdersEquity', 'operatingCashFlow'],
      dtype='object')

In [70]:
def calculate_ttm(df, columns):
    """
    Calculate TTM (Trailing Twelve Months) for specified columns in the DataFrame.
    TTM is computed by summing the values of the most recent quarter + the previous 3 quarters.
    
    Args:
        df (pd.DataFrame): DataFrame containing financial data
        columns (list): List of column names to calculate TTM for
        
    Returns:
        pd.DataFrame: DataFrame with TTM columns added
    """
    # Create a copy to avoid modifying the original dataframe
    df_ttm = df.copy()
    
    # Verify all columns exist in the dataframe
    missing_cols = [col for col in columns if col not in df_ttm.columns]
    if missing_cols:
        raise ValueError(f"Columns not found in dataframe: {missing_cols}")
    
    # Calculate TTM for each column
    for column in columns:
        ttm_values = []
        ttm_column = f'{column}_ttm'
        
        for symbol, group in df_ttm.groupby('symbol'):
            # Sort by date descending to ensure most recent is first
            group = group.sort_values('date', ascending=False)
            ttm = [None] * len(group)
            
            # Calculate TTM for each quarter
            for i in range(len(group) - 3):
                # Sum the latest quarter + previous 3 quarters
                ttm_sum = group.iloc[i:i+4][column].sum()
                ttm[i] = ttm_sum
            
            # Assign TTM values back to the dataframe
            df_ttm.loc[group.index, ttm_column] = ttm
    
    return df_ttm

# Columns to calculate TTM for
ttm_columns = [
    'netIncome',
    'operatingCashFlow',
    'revenue',
    'grossProfit',
    'eps',
    'ebitda'
]

# Create list of TTM column names
ttm_columns_ttm = [f"{col}_ttm" for col in ttm_columns]

try:
    # Apply TTM calculation
    merged_df_with_ttm = calculate_ttm(merged_df, ttm_columns)
    
    # Drop rows where any of the TTM columns are NA
    merged_df_with_ttm = merged_df_with_ttm.dropna(subset=ttm_columns_ttm)
    
    # Print summary information
    print("\nTTM Calculation Summary:")
    print(f"Original number of rows: {len(merged_df)}")
    print(f"Rows after TTM calculation: {len(merged_df_with_ttm)}")
    print(f"Number of rows removed: {len(merged_df) - len(merged_df_with_ttm)}")
    print(f"Number of unique stocks: {merged_df_with_ttm['symbol'].nunique()}")
    print(f"Date range: {merged_df_with_ttm['date'].min()} to {merged_df_with_ttm['date'].max()}")
    
    # Print TTM columns added
    print("\nTTM columns added:")
    for col in ttm_columns_ttm:
        print(f"- {col}")
    
    # Save to CSV
    merged_df_with_ttm.to_csv('merged_financial_data_with_ttm.csv', index=False)
    
    # Display sample data
    print("\nSample data:")
    display(merged_df_with_ttm)

except Exception as e:
    print(f"Error during TTM calculation: {str(e)}")


TTM Calculation Summary:
Original number of rows: 1150
Rows after TTM calculation: 1096
Number of rows removed: 54
Number of unique stocks: 18
Date range: 1996-10-27 00:00:00 to 2024-12-31 00:00:00

TTM columns added:
- netIncome_ttm
- operatingCashFlow_ttm
- revenue_ttm
- grossProfit_ttm
- eps_ttm
- ebitda_ttm

Sample data:


,date,symbol,reportedCurrency,cik,revenue,grossProfit,netIncome,eps,ebitda,weightedAverageShsOut,...,totalDebt,retainedEarnings,totalStockholdersEquity,operatingCashFlow,netIncome_ttm,operatingCashFlow_ttm,revenue_ttm,grossProfit_ttm,eps_ttm,ebitda_ttm
0,2024-12-31,APO,USD,1858681.0,5283000000,4994000000,1.486000e+09,2.4200,2.112000e+09,584000000,...,10588000000,6.022000e+09,1.725300e+10,-4000000,4577000000.0,3253000000,26114000000,23990000000,10.19,8894000000.0
1,2024-12-31,DASH,USD,1792789.0,2873000000,1420000000,1.410000e+08,0.3400,2.580000e+08,417056000,...,536000000,-5.255000e+09,7.803000e+09,518000000,123000000.0,2132000000,10722000000,5040000000,0.29,523000000.0
2,2024-12-31,DECK,USD,910521.0,1827165000,1102623000,4.567340e+08,3.0100,6.017130e+08,151820000,...,257029000,2.424898e+09,2.630919e+09,1095415000,942225000.0,1088841000,4923590000,2827845000,6.186698,1242044724.0
3,2024-12-31,ERIE,USD,922621.0,338709000,338575000,1.520290e+08,3.2600,0.000000e+00,46191602,...,0,3.162303e+09,1.987258e+09,193458000,600314000.0,611249000,3207280000,845266000,12.88,513923000.0
4,2024-12-31,EXE,USD,895126.0,1999000000,592000000,-3.990000e+08,-1.7200,2.900000e+08,231539000,...,5751000000,3.876000e+09,1.756500e+10,382000000,-714000000.0,1561000000,4232000000,907000000,-4.1,1011000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,1996-12-31,DECK,USD,910521.0,22000000,8700000,6.000000e+05,0.0036,2.100000e+06,165446181,...,10400000,2.810000e+07,5.500000e+07,-2200000,3700000.0,8900000,101900000,43100000,0.021934,10200000.0
1131,1996-12-31,ERIE,USD,922621.0,111585056,111585056,2.593236e+07,0.3500,0.000000e+00,74871795,...,0,4.082682e+08,4.357587e+08,28130449,105132359.0,103430449,557285056,557285056,1.21,0.0
1132,1996-12-31,EXE,USD,NaN,71200000,53700000,1.030000e+07,32.0000,4.830000e+07,34638500,...,226800000,5.650000e+07,4.841000e+08,15900000,33455000.0,115772000,211991000,148241000,86.0,143244000.0
1133,1996-12-31,TPL,USD,1811074.0,2681087,2681087,1.564273e+06,0.0365,-1.061440e+06,42857142,...,0,0.000000e+00,1.012566e+07,694438,4364273.0,3294438,8581087,8581087,0.097833,6238560.0


In [68]:
merged_df_with_ttm.columns

Index(['date', 'symbol', 'reportedCurrency', 'cik', 'revenue', 'grossProfit',
       'netIncome', 'eps', 'ebitda', 'weightedAverageShsOut', 'totalAssets',
       'totalCurrentAssets', 'totalLiabilities', 'totalCurrentLiabilities',
       'cashAndCashEquivalents', 'inventory', 'totalDebt', 'retainedEarnings',
       'totalStockholdersEquity', 'operatingCashFlow', 'netIncome_ttm',
       'operatingCashFlow_ttm', 'revenue_ttm', 'grossProfit_ttm', 'eps_ttm',
       'ebitda_ttm'],
      dtype='object')

In [69]:
len(merged_df_with_ttm['symbol'].unique())

18

In [72]:
def get_previous_price_data(df_price, target_date, symbol, max_lookback=5):
    """
    Get the most recent price data before or on the target date for a stock.
    
    Args:
        df_price (pd.DataFrame): DataFrame containing price data
        target_date (pd.Timestamp): Target date to find price for
        symbol (str): Stock symbol
        max_lookback (int): Maximum number of days to look back
    
    Returns:
        pd.Series: Price data (open, high, low, close, volume) or None if not found
    """
    # Reset index to make date and symbol regular columns
    df_price_reset = df_price.reset_index()
    
    # Get price data for the symbol up to target date
    symbol_data = df_price_reset[df_price_reset['symbol'] == symbol]
    symbol_data = symbol_data[symbol_data['date'] <= target_date]
    
    if not symbol_data.empty:
        # Sort by date descending to get most recent first
        symbol_data = symbol_data.sort_values('date', ascending=False)
        
        # Look back up to max_lookback days
        for i in range(min(max_lookback, len(symbol_data))):
            price_data = symbol_data.iloc[i]
            if not price_data[['open', 'high', 'low', 'close', 'volume']].isna().any():
                return price_data[['open', 'high', 'low', 'close', 'volume']]
    
    return None

# Create a copy of merged_df_with_ttm to work with
merged_df_with_ttm_and_price = merged_df_with_ttm.copy()

# Initialize price columns
price_columns = ['open', 'high', 'low', 'close', 'volume']
for col in price_columns:
    merged_df_with_ttm_and_price[col] = None

# Get unique dates and symbols from merged_df_with_ttm
unique_dates = merged_df_with_ttm_and_price['date'].unique()
unique_symbols = merged_df_with_ttm_and_price['symbol'].unique()

# For each date and symbol combination
for date in unique_dates:
    for symbol in unique_symbols:
        # Get the row index for this date and symbol
        mask = (merged_df_with_ttm_and_price['date'] == date) & (merged_df_with_ttm_and_price['symbol'] == symbol)
        if mask.any():
            # Get previous price data
            price_data = get_previous_price_data(combined_df2, date, symbol)
            
            if price_data is not None:
                # Update the price columns
                for col in price_columns:
                    merged_df_with_ttm_and_price.loc[mask, col] = price_data[col]

# Print summary information
print("\nMerged Data Summary:")
print(f"Total rows: {len(merged_df_with_ttm_and_price)}")
print(f"Rows with price data: {merged_df_with_ttm_and_price[price_columns].notna().all(axis=1).sum()}")
print(f"Rows missing price data: {merged_df_with_ttm_and_price[price_columns].isna().any(axis=1).sum()}")
# Print date range
print("\nDate range:")
print(f"From: {merged_df_with_ttm_and_price['date'].min()}")
print(f"To: {merged_df_with_ttm_and_price['date'].max()}")

# Display sample data
print("\nSample data:")
display(merged_df_with_ttm_and_price)

# Save to CSV
merged_df_with_ttm_and_price.to_csv('merged_financial_data_with_ttm_and_price.csv', index=False)


Merged Data Summary:
Total rows: 1096
Rows with price data: 976
Rows missing price data: 120

Date range:
From: 1996-10-27 00:00:00
To: 2024-12-31 00:00:00

Sample data:


,date,symbol,reportedCurrency,cik,revenue,grossProfit,netIncome,eps,ebitda,weightedAverageShsOut,...,operatingCashFlow_ttm,revenue_ttm,grossProfit_ttm,eps_ttm,ebitda_ttm,open,high,low,close,volume
0,2024-12-31,APO,USD,1858681.0,5283000000,4994000000,1.486000e+09,2.4200,2.112000e+09,584000000,...,3253000000,26114000000,23990000000,10.19,8894000000.0,167.0,167.6,164.93,165.16,2817031
1,2024-12-31,DASH,USD,1792789.0,2873000000,1420000000,1.410000e+08,0.3400,2.580000e+08,417056000,...,2132000000,10722000000,5040000000,0.29,523000000.0,169.62,170.08,167.27,167.75,1241521
2,2024-12-31,DECK,USD,910521.0,1827165000,1102623000,4.567340e+08,3.0100,6.017130e+08,151820000,...,1088841000,4923590000,2827845000,6.186698,1242044724.0,206.58,207.0,203.04,203.09,1317389
3,2024-12-31,ERIE,USD,922621.0,338709000,338575000,1.520290e+08,3.2600,0.000000e+00,46191602,...,611249000,3207280000,845266000,12.88,513923000.0,413.83,413.83,410.26,412.23,99000
4,2024-12-31,EXE,USD,895126.0,1999000000,592000000,-3.990000e+08,-1.7200,2.900000e+08,231539000,...,1561000000,4232000000,907000000,-4.1,1011000000.0,99.03,100.27,98.81,99.55,1666085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,1996-12-31,DECK,USD,910521.0,22000000,8700000,6.000000e+05,0.0036,2.100000e+06,165446181,...,8900000,101900000,43100000,0.021934,10200000.0,0.38889,0.40972,0.38194,0.38194,2669418
1131,1996-12-31,ERIE,USD,922621.0,111585056,111585056,2.593236e+07,0.3500,0.000000e+00,74871795,...,103430449,557285056,557285056,1.21,0.0,27.0,31.0,27.0,31.0,900
1132,1996-12-31,EXE,USD,NaN,71200000,53700000,1.030000e+07,32.0000,4.830000e+07,34638500,...,115772000,211991000,148241000,86.0,143244000.0,None,None,None,None,None
1133,1996-12-31,TPL,USD,1811074.0,2681087,2681087,1.564273e+06,0.0365,-1.061440e+06,42857142,...,3294438,8581087,8581087,0.097833,6238560.0,1.81,1.82,1.8,1.82,70500


In [77]:
# Find rows where close price is missing in merged_with_prices
missing_close_prices = merged_df_with_ttm_and_price[merged_df_with_ttm_and_price['close'].isna()]
print(f"Total number of rows with missing close prices: {len(missing_close_prices)}")
print("\nStocks with missing close prices:")
display(missing_close_prices)

Total number of rows with missing close prices: 120

Stocks with missing close prices:


,date,symbol,reportedCurrency,cik,revenue,grossProfit,netIncome,eps,ebitda,weightedAverageShsOut,...,operatingCashFlow_ttm,revenue_ttm,grossProfit_ttm,eps_ttm,ebitda_ttm,open,high,low,close,volume
282,2020-12-31,EXE,USD,895126.0,1207000000,200000000,-4.160000e+08,-42.5300,1.860000e+08,9780547,...,1164000000,4670000000,-347000000,-999.9,-7923000000.0,None,None,None,None,None
298,2020-09-30,EXE,USD,895126.0,1135000000,133000000,-7.450000e+08,-76.1800,-5.500000e+08,9780000,...,1596000000,5515000000,-496000000,-993.37,-7695000000.0,None,None,None,None,None
314,2020-06-30,EXE,USD,895126.0,694000000,-222000000,-2.760000e+08,-28.2200,1.900000e+07,9779000,...,1543000000,6300000000,-643000000,-929.08,-6448000000.0,None,None,None,None,None
318,2020-06-30,PLTR,USD,1321655.0,251889000,183479000,-1.104550e+08,-0.0696,-9.974400e+07,1587519272,...,-61658000,871902000,612597000,-0.5595,-438870000.0,None,None,None,None,None
329,2020-03-31,EXE,USD,895126.0,1634000000,-458000000,-8.297000e+09,-852.9700,-7.578000e+09,9753000,...,1564000000,7717000000,-307000000,-891.65,-5607000000.0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,1998-03-31,EXE,USD,NaN,76800000,68900000,-2.565000e+08,-638.0000,3.680000e+07,74298000,...,137189000,291092000,168195000,-606.0,162293000.0,None,None,None,None,None
1117,1997-09-30,EXE,USD,NaN,72600000,45900000,5.500000e+06,16.0000,3.790000e+07,70376462,...,103889000,285492000,152995000,64.0,173793000.0,None,None,None,None,None
1122,1997-06-30,EXE,USD,NaN,69092000,11395000,-2.177770e+08,0.0000,1.899300e+07,0,...,87289000,261892000,144195000,76.0,168793000.0,None,None,None,None,None
1127,1997-03-31,EXE,USD,NaN,72600000,42000000,5.500000e+06,16.0000,6.860000e+07,69665000,...,105672000,240491000,158241000,76.0,183344000.0,None,None,None,None,None


In [78]:
def check_stock_price(symbol, date):
    """
    Check the close price of a stock on a specific date.
    
    Args:
        symbol (str): Stock symbol
        date (str): Date in 'YYYY-MM-DD' format
    """
    # Convert date to datetime
    check_date = pd.to_datetime(date)
    
    # Get price data for the stock
    stock_data = combined_df2.xs(symbol, level='symbol')
    
    # Try to get the exact date
    try:
        price = stock_data.loc[check_date, 'close']
        print(f"\nClose price for {symbol} on {date}:")
        print(f"Exact date match: ${price:.2f}")
    except KeyError:
        # If exact date not found, get the most recent previous date
        try:
            previous_dates = stock_data.index[stock_data.index <= check_date]
            if len(previous_dates) > 0:
                most_recent_date = previous_dates[-1]
                price = stock_data.loc[most_recent_date, 'close']
                print(f"\nClose price for {symbol} on {date}:")
                print(f"No exact date match found.")
            else:
                print(f"\nNo price data available for {symbol} on or before {date}")
        except Exception as e:
            print(f"\nError finding price data: {str(e)}")

# Example usage:
# Check price for a specific stock and date
symbol = "EXE"
date = "2020-12-31"
check_stock_price(symbol, date)


No price data available for EXE on or before 2020-12-31


In [80]:
# Drop rows with missing close prices
merged_df_with_ttm_and_price = merged_df_with_ttm_and_price.dropna(subset=['close'])

# Print confirmation of the cleaning
print(f"Rows dropped: {len(missing_close_prices)}")
print(f"Remaining rows: {len(merged_df_with_ttm_and_price)}")

Rows dropped: 120
Remaining rows: 976


In [81]:
print(merged_df_with_ttm_and_price['symbol'].nunique()) #no of unique tickers
#len(merged_price_financial['symbol'].unique())

print(len(merged_df_with_ttm_and_price.columns)) #no of columns

18
31


In [82]:
merged_df_with_ttm_and_price

,date,symbol,reportedCurrency,cik,revenue,grossProfit,netIncome,eps,ebitda,weightedAverageShsOut,...,operatingCashFlow_ttm,revenue_ttm,grossProfit_ttm,eps_ttm,ebitda_ttm,open,high,low,close,volume
0,2024-12-31,APO,USD,1858681.0,5283000000,4994000000,1.486000e+09,2.420000,2.112000e+09,584000000,...,3253000000,26114000000,23990000000,10.19,8894000000.0,167.0,167.6,164.93,165.16,2817031
1,2024-12-31,DASH,USD,1792789.0,2873000000,1420000000,1.410000e+08,0.340000,2.580000e+08,417056000,...,2132000000,10722000000,5040000000,0.29,523000000.0,169.62,170.08,167.27,167.75,1241521
2,2024-12-31,DECK,USD,910521.0,1827165000,1102623000,4.567340e+08,3.010000,6.017130e+08,151820000,...,1088841000,4923590000,2827845000,6.186698,1242044724.0,206.58,207.0,203.04,203.09,1317389
3,2024-12-31,ERIE,USD,922621.0,338709000,338575000,1.520290e+08,3.260000,0.000000e+00,46191602,...,611249000,3207280000,845266000,12.88,513923000.0,413.83,413.83,410.26,412.23,99000
4,2024-12-31,EXE,USD,895126.0,1999000000,592000000,-3.990000e+08,-1.720000,2.900000e+08,231539000,...,1561000000,4232000000,907000000,-4.1,1011000000.0,99.03,100.27,98.81,99.55,1666085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,1997-02-02,WSM,USD,719955.0,327700000,153400000,2.550000e+07,0.126377,5.050000e+07,201777778,...,112200000,811800000,339500000,0.112377,65000000.0,3.98,4.02,3.92,3.95,449600
1130,1996-12-31,DECK,USD,910521.0,22000000,8700000,6.000000e+05,0.003600,2.100000e+06,165446181,...,8900000,101900000,43100000,0.021934,10200000.0,0.38889,0.40972,0.38194,0.38194,2669418
1131,1996-12-31,ERIE,USD,922621.0,111585056,111585056,2.593236e+07,0.350000,0.000000e+00,74871795,...,103430449,557285056,557285056,1.21,0.0,27.0,31.0,27.0,31.0,900
1133,1996-12-31,TPL,USD,1811074.0,2681087,2681087,1.564273e+06,0.036500,-1.061440e+06,42857142,...,3294438,8581087,8581087,0.097833,6238560.0,1.81,1.82,1.8,1.82,70500


In [83]:
merged_df_with_ttm_and_price.columns

Index(['date', 'symbol', 'reportedCurrency', 'cik', 'revenue', 'grossProfit',
       'netIncome', 'eps', 'ebitda', 'weightedAverageShsOut', 'totalAssets',
       'totalCurrentAssets', 'totalLiabilities', 'totalCurrentLiabilities',
       'cashAndCashEquivalents', 'inventory', 'totalDebt', 'retainedEarnings',
       'totalStockholdersEquity', 'operatingCashFlow', 'netIncome_ttm',
       'operatingCashFlow_ttm', 'revenue_ttm', 'grossProfit_ttm', 'eps_ttm',
       'ebitda_ttm', 'open', 'high', 'low', 'close', 'volume'],
      dtype='object')

## Calculation of Factors 

In this step, we calculate various financial factors for each ticker using data from the income statement, balance sheet, and cash flow statements. These factors are calculated on a yearly basis and assigned to each day of the corresponding year. Below are the detailed explanations of each factor:

### 1. Quality Factors

- **Net Profit to Total Operating Revenue (TTM)**:
  This ratio is calculated as:
  $$
  \text{Net Profit to Total Operating Revenue (TTM)} = \frac{\text{Net Income}}{\text{Total Revenue}}
  $$
  This measures the profitability of the company relative to its total revenue.

- **Return on Equity (ROE) (TTM)**:
  This ratio is calculated as:
  $$
  \text{ROE (TTM)} = \frac{\text{Net Income}}{\text{Stockholders' Equity}}
  $$
  It indicates how effectively the company is using its equity base to generate profits.

- **Gross Margin Improvement (GMI)**:
  GMI is calculated as the change in gross margin compared to the previous year:
  $$
  \text{GMI} = \left( \frac{\text{Gross Profit}}{\text{Total Revenue}} \right)_{\text{current year}} - \left( \frac{\text{Gross Profit}}{\text{Total Revenue}} \right)_{\text{previous year}}
  $$
  This measures the improvement or decline in gross margin over the previous year.

- **Earnings Manipulation Detection (DECM)**:
  This factor detects potential earnings manipulation by analyzing changes in accounts receivable and inventory relative to total assets:
  $$
  \text{DECM} = \frac{\Delta \text{Accounts Receivable} + \Delta \text{Inventory}}{\text{Total Assets}}
  $$
  It signals potential earnings manipulation if there are significant increases in these components relative to the assets.

- **Accruals Quality Factor (ACCA)**:
  This factor measures the quality of earnings by comparing accruals to actual cash flows:
  $$
  \text{ACCA} = \frac{\text{Net Income} - \text{Cash Flow from Operating Activities}}{\text{Total Assets}}
  $$
  A higher value indicates lower quality of earnings.

### 2. Fundamental Factors

- **Total Liabilities**:
  This factor is directly taken from the balance sheet as:
  $$
  \text{Total Liabilities} = \text{Total Liabilities (Net Minority Interest)}
  $$
  It represents the total amount of liabilities the company holds.

- **Cash Flow to Price Ratio**:
  This ratio is calculated as:
  $$
  \text{Cash Flow to Price Ratio} = \frac{\text{Cash Flow from Continuing Operating Activities} / \text{Shares Outstanding}}{\text{Close Price}}
  $$
  It measures the cash flow generated by the company relative to its stock price.

- **Net Profit (TTM)**:
  This factor represents the net income of the company for the trailing twelve months (TTM):
  $$
  \text{Net Profit (TTM)} = \text{Net Income}
  $$

- **Earnings Before Interest and Taxes (EBIT)**:
  This factor represents the company's earnings before interest and taxes for the year:
  $$
  \text{EBIT} = \text{Earnings Before Interest and Taxes}
  $$

### 3. Growth Factors

- **Net Profit Growth Rate**:
  This factor measures the growth rate of net profit over the previous year:
  $$
  \text{Net Profit Growth Rate} = \frac{\text{Net Income}_{\text{current year}}}{\text{Net Income}_{\text{previous year}}} - 1
  $$

- **Operating Revenue Growth Rate**:
  This factor measures the growth rate of total revenue over the previous year:
  $$
  \text{Operating Revenue Growth Rate} = \frac{\text{Total Revenue}_{\text{current year}}}{\text{Total Revenue}_{\text{previous year}}} - 1
  $$

- **Net Asset Growth Rate**:
  This factor measures the growth rate of stockholders' equity over the previous year:
  $$
  \text{Net Asset Growth Rate} = \frac{\text{Stockholders' Equity}_{\text{current year}}}{\text{Stockholders' Equity}_{\text{previous year}}} - 1
  $$

- **Net Operating Cash Flow Growth Rate**:
  This factor measures the growth rate of cash flow from operating activities over the previous year:
  $$
  \text{Net Operating Cash Flow Growth Rate} = \frac{\text{Cash Flow from Continuing Operating Activities}_{\text{current year}}}{\text{Cash Flow from Continuing Operating Activities}_{\text{previous year}}} - 1
  $$

- **PEG Ratio**:
  The PEG ratio evaluates the stock's valuation by comparing the price-to-earnings (P/E) ratio with the net profit growth rate.
  $$
  \text{PEG} = \frac{\text{P/E Ratio}}{\text{Net Profit Growth Rate}}
  $$
  Here, the P/E Ratio is the ratio of the close price to earnings per share (EPS).

### 4. Stock Factors

- **Net Asset Per Share**:
  This factor is calculated as:
  $$
  \text{Net Asset Per Share} = \frac{\text{Stockholders' Equity}}{\text{Shares Outstanding}}
  $$
  It measures the equity per share held by the company.

- **Net Operating Cash Flow Per Share**:
  This factor is calculated using operating cash flow:
  $$
  \text{Net Operating Cash Flow Per Share} = \frac{\text{Operating Cash Flow}}{\text{Shares Outstanding}}
  $$

- **Cash Flow Per Share (TTM)**:
  This factor is calculated using free cash flow:
  $$
  \text{Cash Flow Per Share (TTM)} = \frac{\text{Free Cash Flow}}{\text{Shares Outstanding}}
  $$

- **Earnings Per Share (TTM)**:
  This factor is calculated as:
  $$
  \text{Earnings Per Share (TTM)} = \frac{\text{Net Income}}{\text{Shares Outstanding}}
  $$

- **Retained Earnings Per Share**:
  This factor is calculated as:
  $$
  \text{Retained Earnings Per Share} = \frac{\text{Retained Earnings}}{\text{Shares Outstanding}}
  $$

### 5. Style Factors

- **Book to Price Ratio**:
  If `Book Value Per Share` is available, it is calculated as:
  $$
  \text{Book to Price Ratio} = \frac{\text{Book Value Per Share}}{\text{Close Price}}
  $$
  If `Book Value Per Share` is not available, the calculation is done as:
  $$
  \text{Book to Price Ratio} = \frac{\text{Common Stock Equity}}{\text{Close Price}}
  $$
  This factor measures how the book value of the company compares to its market price.

- **Size (Market Capitalization)**:
  This factor is calculated as the product of the company's closing price and the number of shares outstanding. It represents the total market value of the company.
  $$
  \text{Market Capitalization} = \text{Close Price} \times \text{Shares Outstanding}
  $$
  This factor was calculated in Step 1 and stored as the `market_cap` column. It is also assigned to the `size` column for analysis.

- **Beta**:
  Beta measures the sensitivity of the stock's returns relative to the S&P 500 index. It is calculated as the covariance of the stock's returns with the market returns, divided by the variance of the market returns. A beta greater than 1 indicates higher sensitivity, while a beta less than 1 indicates lower sensitivity.
  $$
  \beta = \frac{\text{Cov}(\text{R}_{\text{stock}}, \text{R}_{\text{market}})}{\text{Var}(\text{R}_{\text{market}})}
  $$
  where $\text{R}_{\text{stock}}$ represents the daily returns of the stock and $\text{R}_{\text{market}}$ represents the daily returns of the S&P 500 index.

- **Liquidity**:
  Liquidity measures how easily the stock can be traded in the market without impacting its price. It is calculated as the ratio of the trading volume to the number of shares outstanding.
  $$
  \text{Liquidity} = \frac{\text{Volume}}{\text{Shares Outstanding}}
  $$
  Higher liquidity values indicate that the stock is more easily tradable in the market.

- **Momentum**:
  Momentum is calculated as the ratio of the current stock price to the stock price from 12 months ago. It captures the relative change in the stock price over the past year:
  $$
  \text{Momentum} = \frac{\text{Close}_{\text{today}}}{\text{Close}_{\text{252 days ago}}}
  $$
  where $\text{Close}_{\text{252 days ago}}$ is the closing price 252 trading days ago (approximately 1 year). A momentum value greater than 1 indicates that the stock has appreciated over the past year, while a value less than 1 indicates depreciation.

- **Growth**:
  Growth represents the logarithmic growth of the stock price over a 12-month period. It is calculated as the natural logarithm of the ratio between the current stock price and the stock price from 12 months ago:
  $$
  \text{Growth} = \ln \left( \frac{\text{Close}_{\text{today}}}{\text{Close}_{\text{252 days ago}}} \right)
  $$
  This is equivalent to the difference in the natural logarithm of the current stock price and the stock price from 252 trading days ago:
  $$
  \text{Growth} = \ln(\text{Close}_{\text{today}}) - \ln(\text{Close}_{\text{252 days ago}})
  $$
  This factor highlights the exponential growth or decline in the stock's value over the past year.

### 6. Emotional Factors

- **20-Day Volatility (VOL20)**:
  This factor measures the standard deviation of daily returns over the past 20 days, indicating the level of price variability. A higher value represents higher volatility and uncertainty.
  $$
  \text{VOL20} = \sqrt{\frac{\sum_{i=1}^{20} (\text{R}_{i} - \overline{\text{R}})^2}{20}}
  $$
  where $\text{R}_{i}$ represents daily returns and $\overline{\text{R}}$ is the average return over the past 20 days.

- **20-Day Average Volume (DAVOL20)**:
  This factor represents the average trading volume over the past 20 days, reflecting investor interest and liquidity in the stock.
  $$
  \text{DAVOL20} = \frac{\sum_{i=1}^{20} \text{Volume}_{i}}{20}
  $$

- **Volume Oscillator (VOSC)**:
  The volume oscillator calculates the difference between the current volume and its 20-day moving average, providing insight into changes in trading activity.
  $$
  \text{VOSC} = \text{Volume} - \text{DAVOL20}
  $$

- **Volume MACD (VMACD)**:
  This factor is the difference between two exponentially weighted moving averages (12-day and 26-day) of volume, signaling volume momentum.
  $$
  \text{VMACD} = \text{EMA}_{12}(\text{Volume}) - \text{EMA}_{26}(\text{Volume})
  $$

- **Average True Range (ATR14)**:
  ATR14 measures the average range between the high and low prices over the past 14 days, indicating market volatility and potential price movement.
  $$
  \text{ATR14} = \frac{\sum_{i=1}^{14} (\text{High}_{i} - \text{Low}_{i})}{14}
  $$

### 7. Risk Factors

- **20-Day Variance (Variance20)**:
  This factor measures the variance of daily returns over the past 20 days, providing insight into the dispersion of returns.
  $$
  \text{Variance20} = \frac{\sum_{i=1}^{20} (\text{R}_{i} - \overline{\text{R}})^2}{20}
  $$

- **20-Day Sharpe Ratio (sharpe_ratio_20)**:
  The Sharpe ratio measures the excess return per unit of risk (standard deviation) over the past 20 days.
  $$
  \text{Sharpe Ratio}_{20} = \frac{\overline{\text{R}} - \text{R}_{f}}{\text{VOL20}}
  $$
  where $\overline{\text{R}}$ is the average daily return, and $\text{R}_{f}$ is the risk-free rate (assumed to be zero for simplification).

- **20-Day Kurtosis (Kurtosis20)**:
  Kurtosis measures the "tailedness" of the return distribution over the past 20 days. A high value indicates more outliers.
  $$
  \text{Kurtosis}_{20} = \frac{\frac{1}{20} \sum_{i=1}^{20} (\text{R}_{i} - \overline{\text{R}})^4}{\left(\frac{1}{20} \sum_{i=1}^{20} (\text{R}_{i} - \overline{\text{R}})^2\right)^2}
  $$

- **20-Day Skewness (Skewness20)**:
  Skewness measures the asymmetry of the return distribution over the past 20 days. A negative skew indicates a higher probability of negative returns.
  $$
  \text{Skewness}_{20} = \frac{\frac{1}{20} \sum_{i=1}^{20} (\text{R}_{i} - \overline{\text{R}})^3}{\left(\frac{1}{20} \sum_{i=1}^{20} (\text{R}_{i} - \overline{\text{R}})^2\right)^{3/2}}
  $$

- **60-Day Sharpe Ratio (sharpe_ratio_60)**:
  This factor extends the Sharpe ratio calculation to a 60-day period to capture longer-term risk-adjusted performance.
  $$
  \text{Sharpe Ratio}_{60} = \frac{\overline{\text{R}_{60}} - \text{R}_{f}}{\text{VOL60}}
  $$
  where $\overline{\text{R}_{60}}$ is the average return over 60 days, and $\text{VOL60}$ is the standard deviation over 60 days.

### 8. Momentum Factors

- **20-Day Rate of Change (ROC20)**:
  This factor calculates the percentage change in price over the past 20 days, indicating the momentum of the stock.
  $$
  \text{ROC}_{20} = \frac{\text{Close}_{\text{today}} - \text{Close}_{\text{20 days ago}}}{\text{Close}_{\text{20 days ago}}}
  $$

- **One-Month Trading Volume (Volume1M)**:
  This factor sums the trading volume over the past 20 trading days, reflecting the total trading activity in the past month.
  $$
  \text{Volume1M} = \sum_{i=1}^{20} \text{Volume}_{i}
  $$

- **10-Day Triple Exponential Moving Average (TRIX10)**:
  TRIX is a momentum indicator that measures the percentage rate of change of a triple exponentially smoothed moving average. It is calculated using a 10-day period:
  $$
  \text{TRIX}_{10} = \frac{\text{EMA}_{3}(\text{EMA}_{2}(\text{EMA}_{1}(\text{Close})))}{\text{EMA}_{3}(\text{EMA}_{2}(\text{EMA}_{1}(\text{Close}))) - 1}
  $$
  where $\text{EMA}_{1}$, $\text{EMA}_{2}$, and $\text{EMA}_{3}$ are exponentially weighted moving averages with different spans.

- **One-Month Price Change (Price1M)**:
  This factor measures the percentage change in the stock price over the last month (20 trading days).
  $$
  \text{Price1M} = \text{Close}_{\text{today}} - \text{Close}_{\text{20 days ago}}
  $$

- **12-Day Price Momentum (PLRC12)**:
  This factor measures the price momentum by comparing the 12-day moving average with the price 12 days ago:
  $$
  \text{PLRC12} = \frac{\text{MA}_{12}(\text{Close})}{\text{Close}_{\text{12 days ago}}} - 1
  $$

### 9. Technical Factors

- **20-Day MACD (MAC20)**:
  MACD (Moving Average Convergence Divergence) is calculated as the difference between a 12-day and 26-day exponential moving average (EMA) of the close price.
  $$
  \text{MACD}_{20} = \text{EMA}_{12}(\text{Close}) - \text{EMA}_{26}(\text{Close})
  $$

- **Bollinger Bands (boll_up, boll_down)**:
  Bollinger Bands consist of a 20-day simple moving average (SMA) and two bands that are 2 standard deviations away from the SMA.
  $$
  \text{Bollinger Upper Band} = \text{SMA}_{20} + 2 \times \text{VOL20}
  $$
  $$
  \text{Bollinger Lower Band} = \text{SMA}_{20} - 2 \times \text{VOL20}
  $$

- **14-Day Money Flow Index (MFI14)**:
  The MFI14 is a volume-weighted RSI indicator that identifies overbought or oversold conditions.
  1. Calculate the typical price:
  $$
  \text{Typical Price} = \frac{\text{High} + \text{Low} + \text{Close}}{3}
  $$
  2. Calculate raw money flow:
  $$
  \text{Raw Money Flow} = \text{Typical Price} \times \text{Volume}
  $$
  3. Calculate money flow ratio:
  $$
  \text{Money Flow Ratio} = \frac{\sum (\text{Positive Money Flow})}{\sum (\text{Negative Money Flow})}
  $$
  4. Calculate MFI14:
  $$
  \text{MFI14} = 100 - \frac{100}{1 + \text{Money Flow Ratio}}
  $$

After calculating all the factors for each ticker, the data is compiled and saved for further analysis. This enriched dataset can then be used for various machine learning and quantitative analysis applications to evaluate the performance and characteristics of each stock in the S&P 500 index.

In [84]:
#Factor Calculation
print(len(factor_list))

factor_list

54


['net_profit_to_total_revenue_ttm',
 'roe_ttm',
 'roa_ttm',
 'GMI',
 'ACCA',
 'debt_to_asset_ratio',
 'financial_liability',
 'cash_flow_to_price_ratio_ttm',
 'price_to_book_ratio',
 'price_to_sales_ratio_ttm',
 'price_to_earnings_ratio_ttm',
 'total_liability_to_total_asset_ratio',
 'net_profit_ttm',
 'working_capital_ratio',
 'quick_ratio',
 'debt_to_equity_ratio',
 'operate_cash_flow_to_total_asset_ratio',
 'operate_cash_flow_to_total_liabilities_ratio',
 'operate_cash_flow_to_net_profit_ratio',
 'EV_to_operate_cash_flow_ratio',
 'debt_to_EBITDA_ratio',
 'EPS_growth_rate_ttm',
 'PEG_ttm',
 'net_profit_growth_rate_ttm',
 'revenue_growth_rate_ttm',
 'net_asset_growth_rate',
 'operate_cash_flow_growth_rate_ttm',
 'net_asset_per_share',
 'net_operate_cash_flow_per_share_ttm',
 'retained_earnings_per_share',
 'market_cap(size)',
 'VOL60',
 'DAVOL60',
 'VOSC',
 'VMACD',
 'ATR42',
 'ROC60',
 'Volume1Q',
 'TRIX30',
 'Price1Q',
 'PLRC36',
 'Variance60',
 'Skewness60',
 'Kurtosis60',
 'sharpe

In [86]:
# Set pandas option to handle downcasting warning
pd.set_option('future.no_silent_downcasting', True)

#Quality Factors
def calculate_net_profit_to_revenue(stock_data):
    """Calculate net profit to total revenue ratio"""
    if 'netIncome_ttm' in stock_data.columns and 'revenue_ttm' in stock_data.columns:
        return (stock_data['netIncome_ttm'].astype('float64').div(
            stock_data['revenue_ttm'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_roe(stock_data):
    """Calculate Return on Equity"""
    if 'netIncome_ttm' in stock_data.columns and 'totalStockholdersEquity' in stock_data.columns:
        return (stock_data['netIncome_ttm'].astype('float64').div(
            stock_data['totalStockholdersEquity'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_roa(stock_data):
    """Calculate Return on Assets"""
    if 'netIncome_ttm' in stock_data.columns and 'totalAssets' in stock_data.columns:
        stock_data['prev_totalAssets'] = stock_data['totalAssets'].shift(1)
        stock_data['avg_totalAssets'] = ((stock_data['totalAssets'].astype('float64') + 
                                        stock_data['prev_totalAssets'].astype('float64')) / 2)
        return (stock_data['netIncome_ttm'].astype('float64').div(
            stock_data['avg_totalAssets'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_gmi(stock_data):
    """Calculate Gross Margin Index"""
    if 'grossProfit_ttm' in stock_data.columns and 'revenue_ttm' in stock_data.columns:
        stock_data['gross_margin'] = (stock_data['grossProfit_ttm'].astype('float64').div(
            stock_data['revenue_ttm'].replace(0, np.nan).astype('float64')
        ))
        stock_data['prev_gross_margin'] = stock_data['gross_margin'].shift(1)
        return (stock_data['gross_margin'] - stock_data['prev_gross_margin']).astype('float64')
    return None

def calculate_acca(stock_data):
    """Calculate Accruals to Assets"""
    if all(col in stock_data.columns for col in ['netIncome_ttm', 'operatingCashFlow_ttm', 'totalAssets']):
        return ((stock_data['netIncome_ttm'].astype('float64') - 
                stock_data['operatingCashFlow_ttm'].astype('float64')).div(
            stock_data['totalAssets'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_debt_to_asset_ratio(stock_data):
    """Calculate Debt to Asset Ratio"""
    if 'totalDebt' in stock_data.columns and 'totalAssets' in stock_data.columns:
        return (stock_data['totalDebt'].astype('float64').div(
            stock_data['totalAssets'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

#Value Factors
def calculate_financial_liability(stock_data):
    """Calculate financial liability"""
    if 'totalLiabilities' in stock_data.columns:
        return stock_data['totalLiabilities'].astype('float64')
    return None

def calculate_cash_flow_to_price_ratio(stock_data):
    """Calculate cash flow to price ratio"""
    if all(col in stock_data.columns for col in ['operatingCashFlow_ttm', 'weightedAverageShsOut', 'close']):
        return (stock_data['operatingCashFlow_ttm'].astype('float64').div(
            stock_data['weightedAverageShsOut'].replace(0, np.nan).astype('float64')
        ).div(stock_data['close'].replace(0, np.nan).astype('float64'))).astype('float64')
    return None

def calculate_price_to_book_ratio(stock_data):
    """Calculate price to book ratio"""
    if all(col in stock_data.columns for col in ['close', 'totalStockholdersEquity', 'weightedAverageShsOut']):
        return (stock_data['close'].astype('float64').div(
            stock_data['totalStockholdersEquity'].astype('float64').div(
                stock_data['weightedAverageShsOut'].replace(0, np.nan).astype('float64')
            )
        )).astype('float64')
    return None

def calculate_price_to_sales_ratio(stock_data):
    """Calculate price to sales ratio"""
    if all(col in stock_data.columns for col in ['close', 'revenue_ttm', 'weightedAverageShsOut']):
        return (stock_data['close'].astype('float64').div(
            stock_data['revenue_ttm'].astype('float64').div(
                stock_data['weightedAverageShsOut'].replace(0, np.nan).astype('float64')
            )
        )).astype('float64')
    return None

def calculate_price_to_earning_ratio(stock_data):
    """Calculate price to earning ratio"""
    if all(col in stock_data.columns for col in ['close', 'eps_ttm']):
        return (stock_data['close'].astype('float64').div(
            stock_data['eps_ttm'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_total_liability_to_total_asset_ratio(stock_data):
    """Calculate total liability to total asset ratio"""
    if all(col in stock_data.columns for col in ['totalLiabilities', 'totalAssets']):
        return (stock_data['totalLiabilities'].astype('float64').div(
            stock_data['totalAssets'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_net_profit(stock_data):
    """Calculate net profit"""
    if 'netIncome_ttm' in stock_data.columns:
        return stock_data['netIncome_ttm'].astype('float64')
    return None

def calculate_working_capital_ratio(stock_data):
    """Calculate working capital ratio"""
    if all(col in stock_data.columns for col in ['totalCurrentAssets', 'totalCurrentLiabilities']):
        return (stock_data['totalCurrentAssets'].astype('float64').div(
            stock_data['totalCurrentLiabilities'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_quick_ratio(stock_data):
    """Calculate quick ratio"""
    if all(col in stock_data.columns for col in ['totalCurrentAssets', 'inventory', 'totalCurrentLiabilities']):
        return ((stock_data['totalCurrentAssets'].astype('float64') - 
                stock_data['inventory'].astype('float64')).div(
            stock_data['totalCurrentLiabilities'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_debt_to_equity_ratio(stock_data):
    """Calculate debt to equity ratio"""
    if all(col in stock_data.columns for col in ['totalLiabilities', 'totalStockholdersEquity']):
        return (stock_data['totalLiabilities'].astype('float64').div(
            stock_data['totalStockholdersEquity'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_operate_cash_flow_to_total_asset_ratio(stock_data):
    """Calculate operating cash flow to total asset ratio"""
    if all(col in stock_data.columns for col in ['operatingCashFlow_ttm', 'totalAssets']):
        return (stock_data['operatingCashFlow_ttm'].astype('float64').div(
            stock_data['totalAssets'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_operate_cash_flow_to_total_liabilities_ratio(stock_data):
    """Calculate operating cash flow to total liabilities ratio"""
    if all(col in stock_data.columns for col in ['operatingCashFlow_ttm', 'totalLiabilities']):
        return (stock_data['operatingCashFlow_ttm'].astype('float64').div(
            stock_data['totalLiabilities'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_operate_cash_flow_to_net_profit_ratio(stock_data):
    """Calculate operating cash flow to net profit ratio"""
    if all(col in stock_data.columns for col in ['operatingCashFlow_ttm', 'netIncome_ttm']):
        return (stock_data['operatingCashFlow_ttm'].astype('float64').div(
            stock_data['netIncome_ttm'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_EV_to_operate_cash_flow_ratio(stock_data):
    """Calculate EV to operating cash flow ratio"""
    if all(col in stock_data.columns for col in ['close', 'weightedAverageShsOut', 'totalDebt', 'cashAndCashEquivalents', 'operatingCashFlow_ttm']):
        return ((stock_data['close'].astype('float64') * 
                stock_data['weightedAverageShsOut'].astype('float64') + 
                stock_data['totalDebt'].astype('float64') - 
                stock_data['cashAndCashEquivalents'].astype('float64')).div(
            stock_data['operatingCashFlow_ttm'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

def calculate_debt_to_EBITDA_ratio(stock_data):
    """Calculate debt to EBITDA ratio"""
    if all(col in stock_data.columns for col in ['totalDebt', 'ebitda_ttm']):
        return (stock_data['totalDebt'].astype('float64').div(
            stock_data['ebitda_ttm'].replace(0, np.nan).astype('float64')
        )).astype('float64')
    return None

#Growth Factor
def calculate_eps_growth_rate_ttm(stock_data):
    """Calculate EPS growth rate"""
    if 'eps_ttm' in stock_data.columns:
        return (stock_data['eps_ttm'] / stock_data['eps_ttm'].shift(1) - 1).astype('float64')
    return None

def calculate_peg_ttm(stock_data):
    """Calculate PEG ratio"""
    if all(col in stock_data.columns for col in ['close', 'eps_ttm']):
        # Calculate P/E ratio
        pe_ratio = stock_data['close'] / stock_data['eps_ttm']
        # Calculate EPS growth rate
        eps_ttm_growth = (stock_data['eps_ttm'] / stock_data['eps_ttm'].shift(1) - 1)
        # Calculate PEG ratio
        peg_ratio = pe_ratio / eps_ttm_growth.replace(0, np.nan)
        return peg_ratio.astype('float64')
    return None

def calculate_net_profit_growth_rate_ttm(stock_data):
    """Calculate net profit growth rate"""
    if 'netIncome_ttm' in stock_data.columns:
        return (stock_data['netIncome_ttm'] / stock_data['netIncome_ttm'].shift(1) - 1).astype('float64')
    return None

def calculate_revenue_growth_rate_ttm(stock_data):
    """Calculate revenue growth rate"""
    if 'revenue_ttm' in stock_data.columns:
        return (stock_data['revenue_ttm'] / stock_data['revenue_ttm'].shift(1) - 1).astype('float64')
    return None

def calculate_net_asset_growth_rate(stock_data):
    """Calculate net asset growth rate"""
    if 'totalStockholdersEquity' in stock_data.columns:
        return (stock_data['totalStockholdersEquity'] / stock_data['totalStockholdersEquity'].shift(1) - 1).astype('float64')
    return None

def calculate_operate_cash_flow_growth_rate_ttm(stock_data):
    """Calculate operating cash flow growth rate"""
    if 'operatingCashFlow_ttm' in stock_data.columns:
        return (stock_data['operatingCashFlow_ttm'] / stock_data['operatingCashFlow_ttm'].shift(1) - 1).astype('float64')
    return None

#Stock Factor
def calculate_net_asset_per_share(stock_data):
    """Calculate net asset per share"""
    if all(col in stock_data.columns for col in ['totalStockholdersEquity', 'weightedAverageShsOut']):
        return (stock_data['totalStockholdersEquity'] / stock_data['weightedAverageShsOut']).astype('float64')
    return None

def calculate_net_operate_cash_flow_per_share(stock_data):
    """Calculate net operating cash flow per share"""
    if all(col in stock_data.columns for col in ['operatingCashFlow_ttm', 'weightedAverageShsOut']):
        return (stock_data['operatingCashFlow_ttm'] / stock_data['weightedAverageShsOut']).astype('float64')
    return None

def calculate_retained_earnings_per_share(stock_data):
    """Calculate retained earnings per share"""
    if all(col in stock_data.columns for col in ['retainedEarnings', 'weightedAverageShsOut']):
        return (stock_data['retainedEarnings'] / stock_data['weightedAverageShsOut']).astype('float64')
    return None

def calculate_market_cap(stock_data):
    """Calculate market capitalization"""
    if all(col in stock_data.columns for col in ['close', 'weightedAverageShsOut']):
        return (stock_data['close'] * stock_data['weightedAverageShsOut']).astype('float64')
    return None

def calculate_liquidity(stock_data):
    """Calculate liquidity"""
    if all(col in stock_data.columns for col in ['volume', 'weightedAverageShsOut']):
        return (stock_data['volume'] / stock_data['weightedAverageShsOut']).astype('float64')
    return None


# Dictionary mapping factor names to their calculation functions
factor_calculations = {
    #Quality Factors
    'net_profit_to_total_revenue_ttm': calculate_net_profit_to_revenue,
    'roe_ttm': calculate_roe,
    'roa_ttm': calculate_roa,
    'GMI': calculate_gmi,
    'ACCA': calculate_acca,
    'debt_to_asset_ratio': calculate_debt_to_asset_ratio,
    #Value Factors
    'financial_liability': calculate_financial_liability,
    'cash_flow_to_price_ratio_ttm': calculate_cash_flow_to_price_ratio,
    'price_to_book_ratio': calculate_price_to_book_ratio,
    'price_to_sales_ratio_ttm': calculate_price_to_sales_ratio,
    'price_to_earning_ratio_ttm': calculate_price_to_earning_ratio,
    'total_liability_to_total_asset_ratio': calculate_total_liability_to_total_asset_ratio,
    'net_profit_ttm': calculate_net_profit,
    'working_capital_ratio': calculate_working_capital_ratio,
    'quick_ratio': calculate_quick_ratio,
    'debt_to_equity_ratio': calculate_debt_to_equity_ratio,
    'operate_cash_flow_to_total_asset_ratio': calculate_operate_cash_flow_to_total_asset_ratio,
    'operate_cash_flow_to_total_liabilities_ratio': calculate_operate_cash_flow_to_total_liabilities_ratio,
    'operate_cash_flow_to_net_profit_ratio': calculate_operate_cash_flow_to_net_profit_ratio,
    'EV_to_operate_cash_flow_ratio': calculate_EV_to_operate_cash_flow_ratio,
    'debt_to_EBITDA_ratio': calculate_debt_to_EBITDA_ratio,
    #Growth Factor
    'EPS_growth_rate_ttm': calculate_eps_growth_rate_ttm,
    'PEG_ttm': calculate_peg_ttm,
    'net_profit_growth_rate_ttm': calculate_net_profit_growth_rate_ttm,
    'revenue_growth_rate_ttm': calculate_revenue_growth_rate_ttm,
    'net_asset_growth_rate': calculate_net_asset_growth_rate,
    'operate_cash_flow_growth_rate_ttm': calculate_operate_cash_flow_growth_rate_ttm,
    #Stock Factor
    'net_asset_per_share': calculate_net_asset_per_share,
    'net_operate_cash_flow_per_share': calculate_net_operate_cash_flow_per_share,
    'retained_earnings_per_share': calculate_retained_earnings_per_share,
    'market_cap(size)': calculate_market_cap,
    #Style Factor
    'liquidity': calculate_liquidity
}

# Create a copy of the DataFrame
financial_factor_df = merged_df_with_ttm_and_price.copy()

# Add new factor columns with NaN values
for col in factor_calculations.keys():
    financial_factor_df[col] = pd.Series(np.nan, index=financial_factor_df.index, dtype='float64')

# Calculate factors for each stock
for symbol in financial_factor_df['symbol'].unique():
    stock_data = financial_factor_df[financial_factor_df['symbol'] == symbol].sort_values('date')
    
    for factor_name, calc_function in factor_calculations.items():
        try:
            result = calc_function(stock_data)
            if result is not None:
                financial_factor_df.loc[stock_data.index, factor_name] = result
            else:
                print(f"Error: Required columns for {factor_name} not found for {symbol}")
        except Exception as e:
            print(f"Error calculating {factor_name} for {symbol}: {str(e)}")

# Handle any remaining invalid values
for col in factor_calculations.keys():
    financial_factor_df[col] = (
        financial_factor_df[col]
        .replace([np.inf, -np.inf], np.nan)
        .astype('float64')
    )

# Display sample results
print("\nSample results:")

display(financial_factor_df.head())


Sample results:


,date,symbol,reportedCurrency,cik,revenue,grossProfit,netIncome,eps,ebitda,weightedAverageShsOut,...,PEG_ttm,net_profit_growth_rate_ttm,revenue_growth_rate_ttm,net_asset_growth_rate,operate_cash_flow_growth_rate_ttm,net_asset_per_share,net_operate_cash_flow_per_share,retained_earnings_per_share,market_cap(size),liquidity
0,2024-12-31,APO,USD,1858681.0,5283000000,4994000000,1.486000e+09,2.42,2.112000e+09,584000000,...,-87.092167,-0.217205,-0.180789,-0.034149,-0.296040,29.542808,5.570205,10.311644,9.645344e+10,0.004824
1,2024-12-31,DASH,USD,1792789.0,2873000000,1420000000,1.410000e+08,0.34,2.580000e+08,417056000,...,-348.653755,-1.715116,0.056147,0.030916,0.015722,18.709718,5.112023,-12.600226,6.996114e+10,0.002977
2,2024-12-31,DECK,USD,910521.0,1827165000,1102623000,4.567340e+08,3.01,6.017130e+08,151820000,...,391.796466,0.076324,0.057306,0.183372,0.166087,17.329199,7.171921,15.972191,3.083312e+10,0.008677
3,2024-12-31,ERIE,USD,922621.0,338709000,338575000,1.520290e+08,3.26,0.000000e+00,46191602,...,436.437747,0.073498,-0.129931,0.020177,0.079512,43.022063,13.232903,68.460561,1.904156e+10,0.002143
4,2024-12-31,EXE,USD,895126.0,1999000000,592000000,-3.990000e+08,-1.72,2.900000e+08,231539000,...,7.853095,-3.811024,0.047006,0.724087,-0.053366,75.861950,6.741845,16.740160,2.304971e+10,0.007196


In [88]:
#Number of Null Values for Each Factor
financial_factor_df[factor_calculations.keys()].isna().sum()

net_profit_to_total_revenue_ttm                  0
roe_ttm                                          0
roa_ttm                                         18
GMI                                             18
ACCA                                             0
debt_to_asset_ratio                              0
financial_liability                              0
cash_flow_to_price_ratio_ttm                     1
price_to_book_ratio                              1
price_to_sales_ratio_ttm                         1
price_to_earning_ratio_ttm                       1
total_liability_to_total_asset_ratio             0
net_profit_ttm                                   0
working_capital_ratio                           77
quick_ratio                                     77
debt_to_equity_ratio                             0
operate_cash_flow_to_total_asset_ratio           0
operate_cash_flow_to_total_liabilities_ratio     0
operate_cash_flow_to_net_profit_ratio            0
EV_to_operate_cash_flow_ratio  

In [91]:
#Stockwise Null Values
factor_list = list(factor_calculations.keys())
stock_wise_nulls = financial_factor_df.groupby('symbol')[factor_list].apply(lambda x: x.isna().sum())
# Add total null values column
stock_wise_nulls['Total_Null_Values'] = stock_wise_nulls.sum(axis=1)
stock_wise_nulls = stock_wise_nulls.sort_values('Total_Null_Values', ascending=False)

display(stock_wise_nulls)

,net_profit_to_total_revenue_ttm,roe_ttm,roa_ttm,GMI,ACCA,debt_to_asset_ratio,financial_liability,cash_flow_to_price_ratio_ttm,price_to_book_ratio,price_to_sales_ratio_ttm,...,net_profit_growth_rate_ttm,revenue_growth_rate_ttm,net_asset_growth_rate,operate_cash_flow_growth_rate_ttm,net_asset_per_share,net_operate_cash_flow_per_share,retained_earnings_per_share,market_cap(size),liquidity,Total_Null_Values
symbol,,,,,,,,,,,,,,,,,,,,,
ERIE,0,0,1,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,104
TPL,0,0,1,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,68
KKR,0,0,1,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,27
APO,0,0,1,1,0,0,0,1,1,1,...,1,1,1,1,1,1,1,0,1,15
WDAY,0,0,1,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,9
SMCI,0,0,1,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,9
PLTR,0,0,1,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,8
VST,0,0,1,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,8
TKO,0,0,1,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,8


In [92]:
# Print factors for a specific stock and period
factor_df2 = financial_factor_df[['symbol', 'date', 'netIncome_ttm', 'net_profit_ttm', 'operatingCashFlow_ttm'] + list(factor_calculations.keys())]

def print_stock_factors(factor_df2, symbol, start_date=None, end_date=None):
    """
    Print factors for a specific stock and date range
    
    Parameters:
    - factor_df: DataFrame containing the factors
    - symbol: Stock symbol to filter
    - start_date: Start date for filtering (optional)
    - end_date: End date for filtering (optional)
    """
    # Filter for the specific stock
    stock_data = factor_df2[factor_df2['symbol'] == symbol].copy()
    
    # Convert date column to datetime if it's not already
    stock_data['date'] = pd.to_datetime(stock_data['date'])
    
    # Apply date filters if provided
    if start_date:
        stock_data = stock_data[stock_data['date'] >= pd.to_datetime(start_date)]
    if end_date:
        stock_data = stock_data[stock_data['date'] <= pd.to_datetime(end_date)]
    
    # Sort by date
    stock_data = stock_data.sort_values('date', ascending=False)
    
    # Select relevant columns
    #display_columns = ['date'] + factor_columns
    
    print(f"\nFactors for {symbol}:")
    print("=" * 100)
    #print(stock_data[display_columns].to_string())
    print(stock_data.to_string())
    print("=" * 100)

# Example usage:
# Print factors for a specific stock and period
print_stock_factors(factor_df2.round(4), 'DECK', '2023-01-01', '2024-12-31')


Factors for DECK:
    symbol       date netIncome_ttm  net_profit_ttm operatingCashFlow_ttm  net_profit_to_total_revenue_ttm  roe_ttm  roa_ttm     GMI    ACCA  debt_to_asset_ratio  financial_liability  cash_flow_to_price_ratio_ttm  price_to_book_ratio  price_to_sales_ratio_ttm  price_to_earning_ratio_ttm  total_liability_to_total_asset_ratio  net_profit_ttm  working_capital_ratio  quick_ratio  debt_to_equity_ratio  operate_cash_flow_to_total_asset_ratio  operate_cash_flow_to_total_liabilities_ratio  operate_cash_flow_to_net_profit_ratio  EV_to_operate_cash_flow_ratio  debt_to_EBITDA_ratio  EPS_growth_rate_ttm   PEG_ttm  net_profit_growth_rate_ttm  revenue_growth_rate_ttm  net_asset_growth_rate  operate_cash_flow_growth_rate_ttm  net_asset_per_share  net_operate_cash_flow_per_share  retained_earnings_per_share  market_cap(size)  liquidity
2     DECK 2024-12-31   942225000.0     942225000.0            1088841000                           0.1914   0.3581   0.2560  0.0070 -0.0370         

In [93]:
#Save the Quarterly Factors
quarterly_financial_factors = financial_factor_df[['symbol', 'date'] + list(factor_calculations.keys())]
display(quarterly_financial_factors.head())

quarterly_financial_factors.to_csv('quarterly_financial_factors.csv', index=False)

,symbol,date,net_profit_to_total_revenue_ttm,roe_ttm,roa_ttm,GMI,ACCA,debt_to_asset_ratio,financial_liability,cash_flow_to_price_ratio_ttm,...,PEG_ttm,net_profit_growth_rate_ttm,revenue_growth_rate_ttm,net_asset_growth_rate,operate_cash_flow_growth_rate_ttm,net_asset_per_share,net_operate_cash_flow_per_share,retained_earnings_per_share,market_cap(size),liquidity
0,APO,2024-12-31,0.175270,0.265287,0.012261,-0.015583,0.003504,0.028018,3.469150e+11,0.033726,...,-87.092167,-0.217205,-0.180789,-0.034149,-0.296040,29.542808,5.570205,10.311644,9.645344e+10,0.004824
1,DASH,2024-12-31,0.011472,0.015763,0.009792,0.007690,-0.156403,0.041728,5.035000e+09,0.030474,...,-348.653755,-1.715116,0.056147,0.030916,0.015722,18.709718,5.112023,-12.600226,6.996114e+10,0.002977
2,DECK,2024-12-31,0.191370,0.358135,0.255953,0.007040,-0.036984,0.064835,1.333434e+09,0.035314,...,391.796466,0.076324,0.057306,0.183372,0.166087,17.329199,7.171921,15.972191,3.083312e+10,0.008677
3,ERIE,2024-12-31,0.187172,0.302082,0.208705,0.091616,-0.003786,0.000000,9.013560e+08,0.032101,...,436.437747,0.073498,-0.129931,0.020177,0.079512,43.022063,13.232903,68.460561,1.904156e+10,0.002143
4,EXE,2024-12-31,-0.168715,-0.040649,-0.034588,-0.154805,-0.081559,0.206173,1.032900e+10,0.067723,...,7.853095,-3.811024,0.047006,0.724087,-0.053366,75.861950,6.741845,16.740160,2.304971e+10,0.007196


In [94]:
print(len(quarterly_financial_factors.columns))
print(len(quarterly_tech_factors.columns))

34
24


In [96]:
quarterly_financial_factors.columns

Index(['symbol', 'date', 'net_profit_to_total_revenue_ttm', 'roe_ttm',
       'roa_ttm', 'GMI', 'ACCA', 'debt_to_asset_ratio', 'financial_liability',
       'cash_flow_to_price_ratio_ttm', 'price_to_book_ratio',
       'price_to_sales_ratio_ttm', 'price_to_earning_ratio_ttm',
       'total_liability_to_total_asset_ratio', 'net_profit_ttm',
       'working_capital_ratio', 'quick_ratio', 'debt_to_equity_ratio',
       'operate_cash_flow_to_total_asset_ratio',
       'operate_cash_flow_to_total_liabilities_ratio',
       'operate_cash_flow_to_net_profit_ratio',
       'EV_to_operate_cash_flow_ratio', 'debt_to_EBITDA_ratio',
       'EPS_growth_rate_ttm', 'PEG_ttm', 'net_profit_growth_rate_ttm',
       'revenue_growth_rate_ttm', 'net_asset_growth_rate',
       'operate_cash_flow_growth_rate_ttm', 'net_asset_per_share',
       'net_operate_cash_flow_per_share', 'retained_earnings_per_share',
       'market_cap(size)', 'liquidity'],
      dtype='object')

In [95]:
quarterly_dates

DatetimeIndex(['1996-03-31', '1996-06-30', '1996-09-30', '1996-12-31',
               '1997-03-31', '1997-06-30', '1997-09-30', '1997-12-31',
               '1998-03-31', '1998-06-30',
               ...
               '2022-09-30', '2022-12-31', '2023-03-31', '2023-06-30',
               '2023-09-30', '2023-12-31', '2024-03-31', '2024-06-30',
               '2024-09-30', '2024-12-31'],
              dtype='datetime64[ns]', length=116, freq=None)

In [102]:
# First, create a function to map dates to quarter ends
def map_to_quarter_end(date):
    date = pd.to_datetime(date)
    year = date.year
    month = date.month
    
    if 1 <= month <= 3:
        return pd.Timestamp(f"{year}-03-31")
    elif 4 <= month <= 6:
        return pd.Timestamp(f"{year}-06-30")
    elif 7 <= month <= 9:
        return pd.Timestamp(f"{year}-09-30")
    else:  # 10 <= month <= 12
        return pd.Timestamp(f"{year}-12-31")

# Create a copy of quarterly_financial_factors and map its dates
financial_factors_mapped = quarterly_financial_factors.copy()
financial_factors_mapped['mapped_date'] = financial_factors_mapped['date'].apply(map_to_quarter_end)

# Create a copy of quarterly_tech_factors
tech_factors_mapped = quarterly_tech_factors.copy()
tech_factors_mapped['mapped_date'] = tech_factors_mapped['date']

# Merge the DataFrames using the mapped dates
quarterly_merged_factors = pd.merge(
    financial_factors_mapped,
    tech_factors_mapped,
    on=['symbol', 'mapped_date'],
    how='inner',
    suffixes=('_financial', '_tech')
)

# Drop the original date columns and rename mapped_date to date
quarterly_merged_factors = quarterly_merged_factors.drop(['date_financial', 'date_tech'], axis=1)
quarterly_merged_factors = quarterly_merged_factors.rename(columns={'mapped_date': 'date'})

# Sort by symbol and date (latest first)
quarterly_merged_factors = quarterly_merged_factors.sort_values(['symbol', 'date'], ascending=[True, False]).reset_index(drop=True)

columns = quarterly_merged_factors.columns.tolist()
columns.remove('date')
columns = ['date'] + columns

# Reorder the columns
quarterly_merged_factors = quarterly_merged_factors[columns]

# Display results
print("\nFirst few rows of merged factors:")
display(quarterly_merged_factors.head())

print("\nAll quarterly merged factors are saved to 'qaurterly_merged_factors.csv'")
quarterly_merged_factors.to_csv('quarterly_merged_factors.csv', index=False)

print("Shape of merged factors:", quarterly_merged_factors.shape)
# Show the date range
print("\nDate range in merged factors:")
print("Earliest date:", quarterly_merged_factors['date'].min())
print("Latest date:", quarterly_merged_factors['date'].max())

# Check for any missing values
print("\nMissing values in Quarterly Merged Factors:")
display(quarterly_merged_factors.isna().sum())


First few rows of merged factors:


,date,symbol,net_profit_to_total_revenue_ttm,roe_ttm,roa_ttm,GMI,ACCA,debt_to_asset_ratio,financial_liability,cash_flow_to_price_ratio_ttm,...,Kurtosis60,SharpeRatio20,SharpeRatio60,MACD60,boll_up,boll_down,MFI42,GrowthRate,Momentum,Beta
0,2024-12-31,APO,0.175270,0.265287,0.012261,-0.015583,0.003504,0.028018,3.469150e+11,0.033726,...,5.456828,-1.709388,2.514436,12.436933,188.887070,131.343263,74.160994,0.572274,1.772293,1.974168
1,2024-09-30,APO,0.183424,0.327325,0.016351,-0.025489,0.003325,0.026600,3.371480e+11,0.063195,...,6.396609,3.556553,0.714179,1.838533,130.234858,100.120809,48.290705,0.315636,1.371131,1.648160
2,2024-06-30,APO,0.214165,0.374386,0.016804,-0.009987,0.009134,0.028326,3.180900e+11,0.036830,...,0.448297,0.609433,0.779916,3.080801,120.095969,107.004364,59.898434,0.451925,1.571334,1.215277
3,2024-03-31,APO,0.158916,0.369514,0.016876,0.001184,0.000602,0.028726,3.070350e+11,0.079581,...,-0.348075,0.478176,4.263547,5.537078,119.368341,92.161992,55.930718,0.643278,1.902707,0.881649
4,2023-12-31,APO,0.154607,0.359371,0.016916,-0.021476,-0.001834,0.025813,2.882430e+11,0.103302,...,5.997634,0.841613,0.954173,2.688703,96.723526,79.885474,67.204454,0.399154,1.490563,1.161575



All quarterly merged factors are saved to 'qaurterly_merged_factors.csv'
Shape of merged factors: (970, 56)

Date range in merged factors:
Earliest date: 1996-12-31 00:00:00
Latest date: 2024-12-31 00:00:00

Missing values in Quarterly Merged Factors:


date                                             0
symbol                                           0
net_profit_to_total_revenue_ttm                  0
roe_ttm                                          0
roa_ttm                                         12
GMI                                             12
ACCA                                             0
debt_to_asset_ratio                              0
financial_liability                              0
cash_flow_to_price_ratio_ttm                     1
price_to_book_ratio                              1
price_to_sales_ratio_ttm                         1
price_to_earning_ratio_ttm                       1
total_liability_to_total_asset_ratio             0
net_profit_ttm                                   0
working_capital_ratio                           77
quick_ratio                                     77
debt_to_equity_ratio                             0
operate_cash_flow_to_total_asset_ratio           0
operate_cash_flow_to_total_liab

In [103]:
quarterly_merged_factors.columns

Index(['date', 'symbol', 'net_profit_to_total_revenue_ttm', 'roe_ttm',
       'roa_ttm', 'GMI', 'ACCA', 'debt_to_asset_ratio', 'financial_liability',
       'cash_flow_to_price_ratio_ttm', 'price_to_book_ratio',
       'price_to_sales_ratio_ttm', 'price_to_earning_ratio_ttm',
       'total_liability_to_total_asset_ratio', 'net_profit_ttm',
       'working_capital_ratio', 'quick_ratio', 'debt_to_equity_ratio',
       'operate_cash_flow_to_total_asset_ratio',
       'operate_cash_flow_to_total_liabilities_ratio',
       'operate_cash_flow_to_net_profit_ratio',
       'EV_to_operate_cash_flow_ratio', 'debt_to_EBITDA_ratio',
       'EPS_growth_rate_ttm', 'PEG_ttm', 'net_profit_growth_rate_ttm',
       'revenue_growth_rate_ttm', 'net_asset_growth_rate',
       'operate_cash_flow_growth_rate_ttm', 'net_asset_per_share',
       'net_operate_cash_flow_per_share', 'retained_earnings_per_share',
       'market_cap(size)', 'liquidity', 'VOL60', 'DAVOL60', 'VOSC', 'VMACD',
       'ATR42', 'ROC60',

In [100]:
unique_dates = sorted(quarterly_merged_factors['date'].unique(), reverse=True)
for date in unique_dates:
    print(f"{date.strftime('%Y-%m-%d')} - Quarter: Q{(date.month-1)//3 + 1}")

2024-12-31 - Quarter: Q4
2024-09-30 - Quarter: Q3
2024-06-30 - Quarter: Q2
2024-03-31 - Quarter: Q1
2023-12-31 - Quarter: Q4
2023-09-30 - Quarter: Q3
2023-06-30 - Quarter: Q2
2023-03-31 - Quarter: Q1
2022-12-31 - Quarter: Q4
2022-09-30 - Quarter: Q3
2022-06-30 - Quarter: Q2
2022-03-31 - Quarter: Q1
2021-12-31 - Quarter: Q4
2021-09-30 - Quarter: Q3
2021-06-30 - Quarter: Q2
2021-03-31 - Quarter: Q1
2020-12-31 - Quarter: Q4
2020-09-30 - Quarter: Q3
2020-06-30 - Quarter: Q2
2020-03-31 - Quarter: Q1
2019-12-31 - Quarter: Q4
2019-09-30 - Quarter: Q3
2019-06-30 - Quarter: Q2
2019-03-31 - Quarter: Q1
2018-12-31 - Quarter: Q4
2018-09-30 - Quarter: Q3
2018-06-30 - Quarter: Q2
2018-03-31 - Quarter: Q1
2017-12-31 - Quarter: Q4
2017-09-30 - Quarter: Q3
2017-06-30 - Quarter: Q2
2017-03-31 - Quarter: Q1
2016-12-31 - Quarter: Q4
2016-09-30 - Quarter: Q3
2016-06-30 - Quarter: Q2
2016-03-31 - Quarter: Q1
2015-12-31 - Quarter: Q4
2015-09-30 - Quarter: Q3
2015-06-30 - Quarter: Q2
2015-03-31 - Quarter: Q1


In [ ]:
#Stock wise NaN Values in Quarterly_Marged_factors
def analyze_NaN_values(factor_df):
    factor_columns = [col for col in factor_df.columns if col not in ['date', 'symbol']]
    
    # Calculate total NaN values per stock
    stock_nan_totals = factor_df.groupby('symbol')[factor_columns].apply(
        lambda x: x.isna().sum().sum()
    ).reset_index()
    stock_nan_totals.columns = ['Symbol', 'Total NaN Values']
    
    # Sort by total NaN values in descending order
    stock_nan_totals = stock_nan_totals.sort_values('Total NaN Values', ascending=False)
    
    # Calculate detailed statistics for each factor
    detailed_stats = []
    for symbol in factor_df['symbol'].unique():
        stock_data = factor_df[factor_df['symbol'] == symbol]
        
        for col in factor_columns:
            nan_count = stock_data[col].isna().sum()
            total_count = len(stock_data)
            percentage = (nan_count / total_count) * 100
            
            if percentage > 0:  # Only include factors with missing values
                detailed_stats.append({
                    'Symbol': symbol,
                    'Factor Name': col,
                    'Total Values': total_count,
                    'NaN Values': nan_count,
                    'Percentage NaN': f"{percentage:.2f}%"
                })
    
    return stock_nan_totals, pd.DataFrame(detailed_stats)

def print_stock_wise_stats(factor_df):
    # Get statistics
    total_nan_stats, detailed_stats = analyze_NaN_values(factor_df)
    
    # Print summary statistics
    print("\n=== SUMMARY STATISTICS ===")
    print("=" * 100)
    print("\nTotal NaN Values per Stock (Sorted by Missing Values):")
    print(total_nan_stats.to_string(index=False))
    print("\n")
    
    # Print detailed statistics if available
    if not detailed_stats.empty:
        print("=== DETAILED STATISTICS ===")
        print("=" * 100)
        print("\nDetailed Factor-wise Missing Values:")
        print(detailed_stats.sort_values(['Symbol', 'Factor Name']).to_string(index=False))
        
    else:
        print("\nNo stocks have missing factor values!")

# Print stock-wise statistics
print_stock_wise_stats(quarterly_merged_factors)


=== SUMMARY STATISTICS ===

Total NaN Values per Stock (Sorted by Missing Values):
Symbol  Total NaN Values
  ERIE               104
   TPL                68
   KKR                27
  WDAY                15
  SMCI                15
   LII                14
  CRWD                14
   APO                13
  DELL                10
    SW                 8
   TKO                 8
  DECK                 8
   WSM                 8
  GDDY                 6
   EXE                 6
  PLTR                 6
   VST                 6
  DASH                 6


=== DETAILED STATISTICS ===

Detailed Factor-wise Missing Values:
Symbol                       Factor Name  Total Values  NaN Values Percentage NaN
   APO                        GrowthRate            55           3          5.45%
   APO                          Momentum            55           3          5.45%
   APO      cash_flow_to_price_ratio_ttm            55           1          1.82%
   APO                         liquidity     

In [119]:
def print_rows_with_nan_factors(factor_df):
    """
    Print rows that have NaN values in any of the factor columns
    """
    # Get all columns except 'date' and 'symbol'
    factor_columns = [col for col in factor_df.columns if col not in ['date', 'symbol']]
    
    # Find rows with any NaN values in factor columns
    nan_rows = factor_df[factor_df[factor_columns].isna().any(axis=1)]
    
    if not nan_rows.empty:
        print("\nRows with Missing Factor Values:")
        print("=" * 100)
        
        # Sort by symbol and date (latest first)
        nan_rows = nan_rows.sort_values(['symbol', 'date'], ascending=[True, False])
        
        # Display the rows
        print(nan_rows.to_string())
        print("=" * 100)
        
        # Print summary of how many rows have NaN values
        print(f"\nTotal number of rows with missing factors: {len(nan_rows)}")
        
        # Save to CSV
        nan_rows_csv_path = 'rows_with_missing_factors.csv'
        nan_rows.to_csv(nan_rows_csv_path, index=False)
        print(f"\nRows with missing factors saved to: {nan_rows_csv_path}")
    else:
        print("\nNo rows have missing factor values!")

# Print rows with NaN factors
print_rows_with_nan_factors(quarterly_merged_factors)


Rows with Missing Factor Values:
          date symbol  net_profit_to_total_revenue_ttm       roe_ttm   roa_ttm       GMI          ACCA  debt_to_asset_ratio  financial_liability  cash_flow_to_price_ratio_ttm  price_to_book_ratio  price_to_sales_ratio_ttm  price_to_earning_ratio_ttm  total_liability_to_total_asset_ratio  net_profit_ttm  working_capital_ratio  quick_ratio  debt_to_equity_ratio  operate_cash_flow_to_total_asset_ratio  operate_cash_flow_to_total_liabilities_ratio  operate_cash_flow_to_net_profit_ratio  EV_to_operate_cash_flow_ratio  debt_to_EBITDA_ratio  EPS_growth_rate_ttm       PEG_ttm  net_profit_growth_rate_ttm  revenue_growth_rate_ttm  net_asset_growth_rate  operate_cash_flow_growth_rate_ttm  net_asset_per_share  net_operate_cash_flow_per_share  retained_earnings_per_share  market_cap(size)  liquidity     VOL60       DAVOL60          VOSC         VMACD      ATR42     ROC60      Volume1Q    TRIX30    Price1Q    PLRC36  Variance60  Skewness60  Kurtosis60  SharpeRatio20

In [160]:
# Calculate Quarterly Returns
combined_df_return_4 = combined_df.copy()

# Ensure date is datetime
combined_df_return_4['date'] = pd.to_datetime(combined_df_return_4['date'])

# Set date and symbol as index
combined_df_return_4.set_index(['date', 'symbol'], inplace=True)

# Get close prices and unstack to have symbols as columns
close_prices = combined_df_return_4['close'].unstack(level=-1)

# Resample to quarterly frequency and get last price of each quarter
quarterly_prices = close_prices.resample('QE').last()  # Get last price of each quarter

# Calculate percentage change between current quarter and previous quarter
quarterly_returns = quarterly_prices.pct_change().dropna(axis=0, how='all')  # This calculates (current - previous) / previous
quarterly_returns['factor'] = 'quarterly_return'

# Reset index to get date as a column and Melt the dataframe to get it in long format
quarterly_returns = quarterly_returns.reset_index().melt(
    id_vars=['date', 'factor'],
    var_name='symbol',
    value_name='Quarterly_Return'
)

# Pivot the reshaped data back to have 'date' and 'factor' as index and symbols as columns
quarterly_returns = quarterly_returns.pivot(
    index=['date', 'factor'], 
    columns='symbol', 
    values='Quarterly_Return'
)

# Sort by date in descending order (most recent first)
quarterly_returns = quarterly_returns.sort_index(level='date', ascending=False)

# Display the first few rows of quarterly returns
print("Quarterly Returns:")
display(quarterly_returns.head())

# Save to CSV
quarterly_returns.to_csv('quarterly_returns.csv')

Quarterly Returns:


,symbol,APO,CRWD,DASH,DECK,DELL,ERIE,EXE,GDDY,KKR,LII,PLTR,SMCI,SW,TKO,TPL,VST,WDAY,WSM
date,factor,,,,,,,,,,,,,,,,,,
2024-12-31,quarterly_return,0.322232,0.219952,0.175296,0.273691,-0.027839,-0.236357,0.210334,0.258898,0.132716,0.008291,1.033065,-0.268012,0.089842,0.148735,0.250040,0.163067,0.055726,0.195327
2024-09-30,quarterly_return,0.057932,-0.268065,0.312098,-0.011653,-0.140454,0.489570,0.000730,0.122182,0.240783,0.129556,0.468614,-0.491823,-0.016713,0.145569,0.204925,0.378693,0.093264,0.097245
2024-06-30,quarterly_return,0.049978,0.195265,-0.210137,0.028366,0.208571,-0.097542,-0.074750,0.177199,0.046331,0.094566,0.100826,-0.188713,0.016380,0.249740,0.269243,0.234458,-0.180348,-0.110670
2024-03-31,quarterly_return,0.206675,0.255640,0.392659,0.408132,0.491634,0.199003,0.154536,0.117935,0.214001,0.092152,0.340128,2.552585,0.190992,0.059206,0.103711,0.808152,-0.011990,0.573595
2023-12-31,quarterly_return,0.038213,0.525391,0.244369,0.300303,0.110305,0.139998,-0.107735,0.425349,0.344968,0.195171,0.073125,0.036834,0.159777,-0.029503,-0.137698,0.160940,0.284896,0.298456


In [161]:
# Calculate percentage change and shift forward to get next quarter's returns
next_quarter_returns = quarterly_prices.pct_change().dropna(axis=0, how='all')
next_quarter_returns = next_quarter_returns.shift(1)  # Shift forward to get next quarter's returns
next_quarter_returns.dropna(axis=0, how='all', inplace=True)
next_quarter_returns['factor'] = 'next_quarter_return'

# Reset index to get date as a column and Melt the dataframe to get it in long format
next_quarter_returns = next_quarter_returns.reset_index().melt(
    id_vars=['date', 'factor'],
    var_name='symbol',
    value_name='next_quarter_return'
)

# Pivot the reshaped data back to have 'date' and 'factor' as index and symbols as columns
next_quarter_returns = next_quarter_returns.pivot(
    index=['date', 'factor'], 
    columns='symbol', 
    values='next_quarter_return'
)

# Sort by date in descending order (most recent first)
next_quarter_returns = next_quarter_returns.sort_index(level='date', ascending=False)

# Display the first few rows of next quarter returns
print("Next Quarter's Returns:")
display(next_quarter_returns.head())

# Save to CSV
next_quarter_returns.to_csv('next_quarter_returns.csv')

Next Quarter's Returns:


,symbol,APO,CRWD,DASH,DECK,DELL,ERIE,EXE,GDDY,KKR,LII,PLTR,SMCI,SW,TKO,TPL,VST,WDAY,WSM
date,factor,,,,,,,,,,,,,,,,,,
2024-12-31,next_quarter_return,0.057932,-0.268065,0.312098,-0.011653,-0.140454,0.489570,0.000730,0.122182,0.240783,0.129556,0.468614,-0.491823,-0.016713,0.145569,0.204925,0.378693,0.093264,0.097245
2024-09-30,next_quarter_return,0.049978,0.195265,-0.210137,0.028366,0.208571,-0.097542,-0.074750,0.177199,0.046331,0.094566,0.100826,-0.188713,0.016380,0.249740,0.269243,0.234458,-0.180348,-0.110670
2024-06-30,next_quarter_return,0.206675,0.255640,0.392659,0.408132,0.491634,0.199003,0.154536,0.117935,0.214001,0.092152,0.340128,2.552585,0.190992,0.059206,0.103711,0.808152,-0.011990,0.573595
2024-03-31,next_quarter_return,0.038213,0.525391,0.244369,0.300303,0.110305,0.139998,-0.107735,0.425349,0.344968,0.195171,0.073125,0.036834,0.159777,-0.029503,-0.137698,0.160940,0.284896,0.298456
2023-12-31,next_quarter_return,0.168598,0.139647,0.039911,-0.025699,0.273332,0.398933,0.030473,-0.008652,0.100000,0.148342,0.043705,0.099880,0.231510,-0.225039,0.385161,0.264000,-0.048873,0.241809


In [ ]:
# Convert quarterly_merged_factors to pivot format
quarterly_merged_factors = quarterly_merged_factors.reset_index()
quarterly_merged_factors = quarterly_merged_factors.melt(
    id_vars=['date', 'symbol'], 
    var_name='factor', 
    value_name='value'
)
quarterly_merged_factors = quarterly_merged_factors.pivot_table(
    index=['date', 'factor'], 
    columns='symbol', 
    values='value'
)

# 2. Merge all three datasets
combined_quarterly_data = pd.concat([
    quarterly_merged_factors,  # Factor data
    quarterly_returns,         # Current quarter returns
    next_quarter_returns       # Next quarter returns
])

# Sort the data by date (most recent first) and factor
combined_quarterly_data = combined_quarterly_data.sort_index(level=['date', 'factor'], ascending=[False, True])

# Save to CSV
#combined_quarterly_data.to_csv('combined_quarterly_data.csv', index=False)

In [162]:
# Check shape and columns of combined_data
print(f"Shape of combined_data: {combined_quarterly_data.shape}")
print(f"Columns in combined_data: {combined_quarterly_data.columns.tolist()}")
print(f"Start Date: {combined_quarterly_data.index.get_level_values('date').max()}")
print(f"End Date: {combined_quarterly_data.index.get_level_values('date').min()}")
print(f"Index levels in combined_data: {combined_quarterly_data.index.names}")

# Display the first few rows of combined data
print("Combined Quarterly Data:")
display(combined_quarterly_data.head().round(4))

Shape of combined_data: (6324, 18)
Columns in combined_data: ['APO', 'CRWD', 'DASH', 'DECK', 'DELL', 'ERIE', 'EXE', 'GDDY', 'KKR', 'LII', 'PLTR', 'SMCI', 'SW', 'TKO', 'TPL', 'VST', 'WDAY', 'WSM']
Start Date: 2024-12-31 00:00:00
End Date: 1996-03-31 00:00:00
Index levels in combined_data: ['date', 'factor']
Combined Quarterly Data:


symbol                                   APO          CRWD          DASH  \
date       factor                                                          
2024-12-31 ACCA                 3.500000e-03 -1.614000e-01 -1.564000e-01   
           ATR42                4.524000e+00  1.244050e+01  4.538800e+00   
           Beta                 1.974200e+00  2.245600e+00  1.517100e+00   
           DAVOL60              5.337859e+06  3.578626e+06  2.921849e+06   
           EPS_growth_rate_ttm -1.861000e-01 -2.550000e-01 -1.659100e+00   

symbol                                  DECK          DELL         ERIE  \
date       factor                                                         
2024-12-31 ACCA                -3.700000e-02 -1.680000e-02      -0.0038   
           ATR42                5.108100e+00  4.246900e+00      10.2995   
           Beta                 1.308200e+00  1.766000e+00       1.2313   
           DAVOL60              1.920348e+06  7.455264e+06  147967.2000   
           EPS_growth_rate_ttm  8.380000e-02  3.410000e-02       0.0733   

symbol                                   EXE          GDDY           KKR  \
date       factor                                                          
2024-12-31 ACCA                -8.160000e-02 -4.260000e-02 -9.900000e-03   
           ATR42                2.151200e+00  4.412600e+00  3.663100e+00   
           Beta                 9.692000e-01  1.084100e+00  1.984500e+00   
           DAVOL60              2.497039e+06  1.194428e+06  3.165574e+06   
           EPS_growth_rate_ttm -3.091800e+00 -4.996000e-01  2.660000e-02   

symbol                                  LII          PLTR          SMCI  \
date       factor                                                         
2024-12-31 ACCA                     -0.0400 -1.091000e-01  3.533000e-01   
           ATR42                    15.9714  3.864400e+00  3.034500e+00   
           Beta                      0.7537  2.644800e+00  1.436700e+00   
           DAVOL60              381805.5667  7.467036e+07  8.116139e+07   
           EPS_growth_rate_ttm       0.0704 -5.930000e-02 -3.900000e-03   

symbol                                    SW          TKO          TPL  \
date       factor                                                        
2024-12-31 ACCA                -4.810000e-02      -0.0452      -0.0294   
           ATR42                1.213100e+00       3.5588      70.4910   
           Beta                 4.055000e-01       0.6618       1.2919   
           DAVOL60              3.626387e+06  924131.4500  228254.8667   
           EPS_growth_rate_ttm -5.907000e-01      27.0000       0.0121   

symbol                                   VST          WDAY           WSM  
date       factor                                                         
2024-12-31 ACCA                -5.040000e-02 -4.410000e-02 -6.180000e-02  
           ATR42                7.419300e+00  7.731400e+00  5.388300e+00  
           Beta                 2.076000e+00  1.278500e+00  4.846000e-01  
           DAVOL60              6.079748e+06  3.265368e+06  2.067788e+06  
           EPS_growth_rate_ttm  3.514000e-01  5.130000e-02  1.660000e-02

In [164]:
# Drop the 'index' factor from the DataFrame
combined_quarterly_data = combined_quarterly_data[combined_quarterly_data.index.get_level_values(1) != 'index']

# Print factors to verify the change
print("\nFactors in the Dataset:")
print("=" * 50)

# Get unique factors from the index
factors = combined_quarterly_data.index.get_level_values(1).unique()

# Print each factor on a new line with numbering
for i, factor in enumerate(factors, 1):
    print(f"{i}. {factor}")

print("=" * 50)
print(f"Total number of factors: {len(factors)}")


Factors in the Dataset:
1. ACCA
2. ATR42
3. Beta
4. DAVOL60
5. EPS_growth_rate_ttm
6. EV_to_operate_cash_flow_ratio
7. GMI
8. GrowthRate
9. Kurtosis60
10. MACD60
11. MFI42
12. Momentum
13. PEG_ttm
14. PLRC36
15. Price1Q
16. ROC60
17. SharpeRatio20
18. SharpeRatio60
19. Skewness60
20. TRIX30
21. VMACD
22. VOL60
23. VOSC
24. Variance60
25. Volume1Q
26. boll_down
27. boll_up
28. cash_flow_to_price_ratio_ttm
29. debt_to_EBITDA_ratio
30. debt_to_asset_ratio
31. debt_to_equity_ratio
32. financial_liability
33. liquidity
34. market_cap(size)
35. net_asset_growth_rate
36. net_asset_per_share
37. net_operate_cash_flow_per_share
38. net_profit_growth_rate_ttm
39. net_profit_to_total_revenue_ttm
40. net_profit_ttm
41. next_quarter_return
42. operate_cash_flow_growth_rate_ttm
43. operate_cash_flow_to_net_profit_ratio
44. operate_cash_flow_to_total_asset_ratio
45. operate_cash_flow_to_total_liabilities_ratio
46. price_to_book_ratio
47. price_to_earning_ratio_ttm
48. price_to_sales_ratio_ttm
49. qu

In [165]:
# Display the first few rows to verify the format
print("First few rows of Combined Quartely Data:")
display(combined_quarterly_data.head().round(4))

# Save the result to a CSV file
print(f"Combined Quarterly Data saved to {'combined_quarterly_data.csv'}")
combined_quarterly_data.to_csv('combined_quarterly_data.csv')

First few rows of Combined Quartely Data:


symbol                                   APO          CRWD          DASH  \
date       factor                                                          
2024-12-31 ACCA                 3.500000e-03 -1.614000e-01 -1.564000e-01   
           ATR42                4.524000e+00  1.244050e+01  4.538800e+00   
           Beta                 1.974200e+00  2.245600e+00  1.517100e+00   
           DAVOL60              5.337859e+06  3.578626e+06  2.921849e+06   
           EPS_growth_rate_ttm -1.861000e-01 -2.550000e-01 -1.659100e+00   

symbol                                  DECK          DELL         ERIE  \
date       factor                                                         
2024-12-31 ACCA                -3.700000e-02 -1.680000e-02      -0.0038   
           ATR42                5.108100e+00  4.246900e+00      10.2995   
           Beta                 1.308200e+00  1.766000e+00       1.2313   
           DAVOL60              1.920348e+06  7.455264e+06  147967.2000   
           EPS_growth_rate_ttm  8.380000e-02  3.410000e-02       0.0733   

symbol                                   EXE          GDDY           KKR  \
date       factor                                                          
2024-12-31 ACCA                -8.160000e-02 -4.260000e-02 -9.900000e-03   
           ATR42                2.151200e+00  4.412600e+00  3.663100e+00   
           Beta                 9.692000e-01  1.084100e+00  1.984500e+00   
           DAVOL60              2.497039e+06  1.194428e+06  3.165574e+06   
           EPS_growth_rate_ttm -3.091800e+00 -4.996000e-01  2.660000e-02   

symbol                                  LII          PLTR          SMCI  \
date       factor                                                         
2024-12-31 ACCA                     -0.0400 -1.091000e-01  3.533000e-01   
           ATR42                    15.9714  3.864400e+00  3.034500e+00   
           Beta                      0.7537  2.644800e+00  1.436700e+00   
           DAVOL60              381805.5667  7.467036e+07  8.116139e+07   
           EPS_growth_rate_ttm       0.0704 -5.930000e-02 -3.900000e-03   

symbol                                    SW          TKO          TPL  \
date       factor                                                        
2024-12-31 ACCA                -4.810000e-02      -0.0452      -0.0294   
           ATR42                1.213100e+00       3.5588      70.4910   
           Beta                 4.055000e-01       0.6618       1.2919   
           DAVOL60              3.626387e+06  924131.4500  228254.8667   
           EPS_growth_rate_ttm -5.907000e-01      27.0000       0.0121   

symbol                                   VST          WDAY           WSM  
date       factor                                                         
2024-12-31 ACCA                -5.040000e-02 -4.410000e-02 -6.180000e-02  
           ATR42                7.419300e+00  7.731400e+00  5.388300e+00  
           Beta                 2.076000e+00  1.278500e+00  4.846000e-01  
           DAVOL60              6.079748e+06  3.265368e+06  2.067788e+06  
           EPS_growth_rate_ttm  3.514000e-01  5.130000e-02  1.660000e-02

Combined Quarterly Data saved to combined_quarterly_data.csv
